# Load data and initial cleaning

## Load all data sets and drop unneded variables

### Get inferred ancestry dataset (Python kernel)

In [ ]:
# mkdir ./ancestry

In [ ]:
# !gsutil -u $GOOGLE_PROJECT -m cp gs://fc-aou-datasets-controlled/v8/wgs/short_read/snpindel/aux/ancestry/ancestry_preds.tsv \ancestry

### Get sample QC information dataset (Python kernel)

In [ ]:
# mkdir ./sample_QC

In [ ]:
# !gsutil -u $GOOGLE_PROJECT -m cp gs://fc-aou-datasets-controlled/v8/wgs/short_read/snpindel/aux/qc/genomic_metrics.tsv \sample_QC

### Prepare the core data sets 

In [ ]:
library(tidyverse)
library(bigrquery)

# # This query represents dataset "All srWGS cancer data" for domain "person" and was generated for All of Us Controlled Tier Dataset v8
# dataset_79764376_person_sql <- paste("
#     SELECT
#         person.person_id,
#         p_gender_concept.concept_name as gender,
#         person.birth_datetime as date_of_birth,
#         p_race_concept.concept_name as race,
#         p_ethnicity_concept.concept_name as ethnicity,
#         p_sex_at_birth_concept.concept_name as sex_at_birth 
#     FROM
#         `person` person 
#     LEFT JOIN
#         `concept` p_gender_concept 
#             ON person.gender_concept_id = p_gender_concept.concept_id 
#     LEFT JOIN
#         `concept` p_race_concept 
#             ON person.race_concept_id = p_race_concept.concept_id 
#     LEFT JOIN
#         `concept` p_ethnicity_concept 
#             ON person.ethnicity_concept_id = p_ethnicity_concept.concept_id 
#     LEFT JOIN
#         `concept` p_sex_at_birth_concept 
#             ON person.sex_at_birth_concept_id = p_sex_at_birth_concept.concept_id  
#     WHERE
#         person.PERSON_ID IN (SELECT
#             distinct person_id  
#         FROM
#             `cb_search_person` cb_search_person  
#         WHERE
#             cb_search_person.person_id IN (SELECT
#                 person_id 
#             FROM
#                 `cb_search_person` p 
#             WHERE
#                 has_ehr_data = 1 ) 
#             AND cb_search_person.person_id IN (SELECT
#                 person_id 
#             FROM
#                 `cb_search_person` p 
#             WHERE
#                 has_whole_genome_variant = 1 ) 
#             AND cb_search_person.person_id IN (SELECT
#                 criteria.person_id 
#             FROM
#                 (SELECT
#                     DISTINCT person_id, entry_date, concept_id 
#                 FROM
#                     `cb_search_all_events` 
#                 WHERE
#                     (concept_id IN(SELECT
#                         DISTINCT c.concept_id 
#                     FROM
#                         `cb_criteria` c 
#                     JOIN
#                         (SELECT
#                             CAST(cr.id as string) AS id       
#                         FROM
#                             `cb_criteria` cr       
#                         WHERE
#                             concept_id IN (35206491, 44836821, 44828755, 35206457, 44820669, 1567641, 44824027, 44822872, 1567492, 44828775, 1567651, 44827551, 35206249, 44826387, 1567478, 44827552, 1567462, 44831033, 1567729, 44827586, 44834505, 1567461, 44827555, 44827554, 44833291, 44832117, 35206466, 1567586, 44826401, 44825211, 44828741, 35206458, 1567584, 1567674, 44829811, 1567515, 1567721, 44826398, 44833294, 1567537, 44832129, 44820621, 44825192, 1567530, 35206139, 44819440, 44834496, 44828731, 35206153, 35206140, 1567581, 35206252, 44829795, 1567582, 44825196, 44825200, 44831017, 45581270, 1567597, 44829838, 44835663, 44835667, 1567722, 44833313, 44832131, 44826394, 44819488, 45595728, 44825199, 44832180, 44819422, 1567469, 44836825, 44820604, 44829814, 44830987, 44826392, 1567659, 1567668, 35206459, 1567479, 44828776, 44835690, 35206131, 35206266, 35206461, 1567502, 44836826, 44829802, 44836845, 44830972, 1567473, 44834480, 35206248, 44826449, 44827579, 44825214, 44830991,
#  1567574, 44834482, 1567568, 44835662, 45605336, 35206185, 44835735, 44820605, 35206247, 44826450, 35206137, 44822889, 45581269, 1567483, 44827547, 45547562, 1567529, 1567680, 44830997, 44829789, 1567735, 44826391, 44833300, 44824034, 1567494, 44833283, 44835673, 44828750, 44821756, 44831036, 44830967, 35206242, 44824068, 44826409, 35206183, 35206133, 35206260, 1567675, 44836830, 44820600, 1567501, 1567466, 44825195, 35206492, 1567463, 35206304, 44821775, 35206132, 1567471, 44833301, 1567746, 44832181, 44822887, 35206134, 44826420, 44836846, 44836837, 44828729, 35206467, 35206251, 44833282, 44825252, 44829788, 1567699, 1567596, 44833287, 1567470, 44836831, 35206138, 35206056, 44833292, 1567534, 35206250, 1567472, 44830971, 44819425, 1567652, 44819434, 44829800, 44824023, 1567740, 1567476, 1567689, 44834486, 44819421, 44835668, 44828733, 1567573, 44825251, 44833356, 1567583, 44830968, 35206489, 35206253, 44835665, 44835712, 44832148, 1567633, 1567474, 44827556, 35206101, 44836833,
#  44835677, 44833293, 44834535, 44827545, 1567468, 44826393, 44833306, 44833334, 1567673, 1567485, 35206130, 44832119, 44833284, 44825194, 1567705, 44827544, 44821758, 44822893, 44825208, 44827563, 35206135, 44820628, 1567484, 44824020, 44822870, 1567598, 44828744, 44826399, 35206493, 35206080, 44820616, 44825191, 1567464, 44836844, 44820617, 35206494, 35206490, 1567465, 35206136, 44829815, 44825232, 1567603, 35206141, 1567528, 44822886, 44826389, 35206462, 44835671, 1567475, 1567481, 44835693, 44827548, 44827546, 44824024, 1567566, 1567486, 44832122, 44826386, 44826413, 1567565, 1567533, 44832179, 44830969)       
#                             AND full_text LIKE '%_rank1]%'      ) a 
#                             ON (c.path LIKE CONCAT('%.', a.id, '.%') 
#                             OR c.path LIKE CONCAT('%.', a.id) 
#                             OR c.path LIKE CONCAT(a.id, '.%') 
#                             OR c.path = a.id) 
#                     WHERE
#                         is_standard = 0 
#                         AND is_selectable = 1) 
#                     AND is_standard = 0 )) criteria )  
#         UNION
#         DISTINCT SELECT
#             distinct person_id  
#         FROM
#             `cb_search_person` cb_search_person  
#         WHERE
#             cb_search_person.person_id IN (SELECT
#                 person_id 
#             FROM
#                 `cb_search_person` p 
#             WHERE
#                 has_ehr_data = 1 ) 
#             AND cb_search_person.person_id IN (SELECT
#                 person_id 
#             FROM
#                 `cb_search_person` p 
#             WHERE
#                 has_whole_genome_variant = 1 ) 
#             AND cb_search_person.person_id NOT IN (SELECT
#                 criteria.person_id 
#             FROM
#                 (SELECT
#                     DISTINCT person_id, entry_date, concept_id 
#                 FROM
#                     `cb_search_all_events` 
#                 WHERE
#                     (concept_id IN(SELECT
#                         DISTINCT c.concept_id 
#                     FROM
#                         `cb_criteria` c 
#                     JOIN
#                         (SELECT
#                             CAST(cr.id as string) AS id       
#                         FROM
#                             `cb_criteria` cr       
#                         WHERE
#                             concept_id IN (35206491, 44836821, 44828755, 35206457, 44820669, 1567641, 44824027, 44822872, 1567492, 44828775, 1567651, 44827551, 35206249, 44826387, 1567478, 44827552, 1567462, 44831033, 1567729, 44827586, 44834505, 1567461, 44827555, 44827554, 44833291, 44832117, 35206466, 1567586, 44826401, 44825211, 44828741, 35206458, 1567584, 1567674, 44829811, 1567515, 1567721, 44826398, 44833294, 1567537, 44832129, 44820621, 44825192, 1567530, 35206139, 44819440, 44834496, 44828731, 35206153, 35206140, 1567581, 35206252, 44829795, 1567582, 44825196, 44825200, 44831017, 45581270, 1567597, 44829838, 44835663, 44835667, 1567722, 44833313, 44832131, 44826394, 44819488, 45595728, 44825199, 44832180, 44819422, 1567469, 44836825, 44820604, 44829814, 44830987, 44826392, 1567659, 1567668, 35206459, 1567479, 44828776, 44835690, 35206131, 35206266, 35206461, 1567502, 44836826, 44829802, 44836845, 44830972, 1567473, 44834480, 35206248, 44826449, 44827579, 44825214, 44830991,
#  1567574, 44834482, 1567568, 44835662, 45605336, 35206185, 44835735, 44820605, 35206247, 44826450, 35206137, 44822889, 45581269, 1567483, 44827547, 45547562, 1567529, 1567680, 44830997, 44829789, 1567735, 44826391, 44833300, 44824034, 1567494, 44833283, 44835673, 44828750, 44821756, 44831036, 44830967, 35206242, 44824068, 44826409, 35206183, 35206133, 35206260, 1567675, 44836830, 44820600, 1567501, 1567466, 44825195, 35206492, 1567463, 35206304, 44821775, 35206132, 1567471, 44833301, 1567746, 44832181, 44822887, 35206134, 44826420, 44836846, 44836837, 44828729, 35206467, 35206251, 44833282, 44825252, 44829788, 1567699, 1567596, 44833287, 1567470, 44836831, 35206138, 35206056, 44833292, 1567534, 35206250, 1567472, 44830971, 44819425, 1567652, 44819434, 44829800, 44824023, 1567740, 1567476, 1567689, 44834486, 44819421, 44835668, 44828733, 1567573, 44825251, 44833356, 1567583, 44830968, 35206489, 35206253, 44835665, 44835712, 44832148, 1567633, 1567474, 44827556, 35206101, 44836833,
#  44835677, 44833293, 44834535, 44827545, 1567468, 44826393, 44833306, 44833334, 1567673, 1567485, 35206130, 44832119, 44833284, 44825194, 1567705, 44827544, 44821758, 44822893, 44825208, 44827563, 35206135, 44820628, 1567484, 44824020, 44822870, 1567598, 44828744, 44826399, 35206493, 35206080, 44820616, 44825191, 1567464, 44836844, 44820617, 35206494, 35206490, 1567465, 35206136, 44829815, 44825232, 1567603, 35206141, 1567528, 44822886, 44826389, 35206462, 44835671, 1567475, 1567481, 44835693, 44827548, 44827546, 44824024, 1567566, 1567486, 44832122, 44826386, 44826413, 1567565, 1567533, 44832179, 44830969)       
#                             AND full_text LIKE '%_rank1]%'      ) a 
#                             ON (c.path LIKE CONCAT('%.', a.id, '.%') 
#                             OR c.path LIKE CONCAT('%.', a.id) 
#                             OR c.path LIKE CONCAT(a.id, '.%') 
#                             OR c.path = a.id) 
#                     WHERE
#                         is_standard = 0 
#                         AND is_selectable = 1) 
#                     AND is_standard = 0 )) criteria ) 
#             AND cb_search_person.person_id NOT IN (SELECT
#                 criteria.person_id 
#             FROM
#                 (SELECT
#                     DISTINCT person_id, entry_date, concept_id 
#                 FROM
#                     `cb_search_all_events` 
#                 WHERE
#                     (concept_id IN (43021268, 4324321, 46273375, 46270536, 4201829, 46273407, 35610791, 46273418, 46270077, 762250, 4187203, 4178769, 43021272, 4333345, 4176919, 4325868, 4212564, 4190635, 4324190, 4327107, 4179242, 4190634, 4212563, 46273480, 4187206, 4180749, 44782983, 4181024, 37016185, 4327885, 4325851, 4190633, 46273372, 4324189, 4324191, 4178782, 40483717, 4179084, 4187205, 4333465, 43021269, 4177063, 4216132, 4190632, 4180131, 4180113, 4197758, 4144289, 43021271, 4212565, 37016142, 45763741, 37018569, 4179069, 46274037) 
#                     AND is_standard = 1 )) criteria ) )", sep="")

# # Formulate a Cloud Storage destination path for the data exported from BigQuery.
# # NOTE: By default data exported multiple times on the same day will overwrite older copies.
# #       But data exported on a different days will write to a new location so that historical
# #       copies can be kept as the dataset definition is changed.
# person_79764376_path <- file.path(
#   Sys.getenv("WORKSPACE_BUCKET"),
#   "AoU_cancer",
#   "phenotypes",
#   "person_cancer_data.csv")
# message(str_glue('The data will be written to {person_79764376_path}. Use this path when reading ',
#                  'the data into your notebooks in the future.'))

# # Perform the query and export the dataset to Cloud Storage as CSV files.
# # NOTE: You only need to run `bq_table_save` once. After that, you can
# #       just read data from the CSVs in Cloud Storage.
# bq_table_save(
#   bq_dataset_query(Sys.getenv("WORKSPACE_CDR"), dataset_79764376_person_sql, billing = Sys.getenv("GOOGLE_PROJECT")),
#   person_79764376_path,
#   destination_format = "CSV")


# # This query represents dataset "All srWGS cancer data" for domain "condition" and was generated for All of Us Controlled Tier Dataset v8
# dataset_79764376_condition_sql <- paste("
#     SELECT
#         c_occurrence.person_id,
#         c_standard_concept.concept_name as standard_concept_name,
#         c_standard_concept.concept_code as standard_concept_code,
#         c_standard_concept.vocabulary_id as standard_vocabulary,
#         c_occurrence.condition_start_datetime,
#         c_occurrence.condition_end_datetime,
#         c_type.concept_name as condition_type_concept_name,
#         c_occurrence.stop_reason,
#         visit.concept_name as visit_occurrence_concept_name,
#         c_occurrence.condition_source_value,
#         c_source_concept.concept_name as source_concept_name,
#         c_source_concept.concept_code as source_concept_code,
#         c_source_concept.vocabulary_id as source_vocabulary,
#         c_occurrence.condition_status_source_value,
#         c_status.concept_name as condition_status_concept_name 
#     FROM
#         ( SELECT
#             * 
#         FROM
#             `condition_occurrence` c_occurrence 
#         WHERE
#             (
#                 condition_source_concept_id IN (SELECT
#                     DISTINCT c.concept_id 
#                 FROM
#                     `cb_criteria` c 
#                 JOIN
#                     (SELECT
#                         CAST(cr.id as string) AS id       
#                     FROM
#                         `cb_criteria` cr       
#                     WHERE
#                         concept_id IN (1567470, 1567474, 1567475, 1567479, 1567485, 1567486, 1567490, 1567491, 1567502, 1567515, 1567537, 1567563, 1567564, 1567566, 1567568, 1567574, 1567577, 1567581, 1567584, 1567596, 1567597, 1567603, 1567633, 1567640, 1567641, 1567642, 1567652, 1567653, 1567654, 1567656, 1567660, 1567661, 1567672, 1567675, 1567676, 1567681, 1567682, 1567690, 1567691, 1567694, 1567707, 1567722, 1567729, 1567736, 1567738, 1567740, 1595483, 1595484, 1595486, 1595491, 1595492, 1595493, 1595494, 1595495, 1595496, 1595497, 1595498, 1595499, 1595500, 1595501, 1595502, 1595504, 1595506, 1595509, 1595510, 1595511, 1595512, 1595513, 1595514, 1595516, 1595517, 1595518, 1595519, 1595520, 1595521, 1595522, 35206047, 35206048, 35206049, 35206050, 35206051, 35206053, 35206054, 35206055, 35206056, 35206057, 35206058, 35206059, 35206060, 35206061, 35206062, 35206063, 35206064, 35206065, 35206066, 35206067, 35206068, 35206069, 35206070, 35206071, 35206072, 35206073, 35206074,
#  35206075, 35206076, 35206077, 35206078, 35206079, 35206080, 35206081, 35206082, 35206083, 35206084, 35206085, 35206086, 35206087, 35206088, 35206089, 35206090, 35206091, 35206092, 35206093, 35206094, 35206095, 35206096, 35206097, 35206098, 35206099, 35206100, 35206101, 35206102, 35206103, 35206104, 35206105, 35206106, 35206107, 35206109, 35206110, 35206111, 35206112, 35206113, 35206114, 35206115, 35206116, 35206117, 35206118, 35206119, 35206120, 35206121, 35206122, 35206123, 35206124, 35206125, 35206126, 35206127, 35206128, 35206129, 35206130, 35206131, 35206132, 35206133, 35206134, 35206135, 35206136, 35206137, 35206138, 35206139, 35206140, 35206141, 35206142, 35206143, 35206144, 35206145, 35206146, 35206147, 35206148, 35206149, 35206150, 35206151, 35206152, 35206153, 35206158, 35206159, 35206160, 35206161, 35206162, 35206163, 35206164, 35206165, 35206169, 35206170, 35206171, 35206172, 35206173, 35206174, 35206175, 35206176, 35206177, 35206178, 35206179, 35206180, 35206181, 35206182,
#  35206183, 35206184, 35206185, 35206186, 35206187, 35206188, 35206189, 35206190, 35206191, 35206194, 35206195, 35206196, 35206197, 35206198, 35206199, 35206200, 35206201, 35206202, 35206203, 35206208, 35206209, 35206210, 35206211, 35206212, 35206213, 35206214, 35206215, 35206216, 35206217, 35206218, 35206219, 35206220, 35206221, 35206222, 35206223, 35206224, 35206225, 35206226, 35206227, 35206228, 35206229, 35206230, 35206231, 35206232, 35206233, 35206234, 35206235, 35206236, 35206237, 35206238, 35206239, 35206240, 35206241, 35206242, 35206243, 35206244, 35206245, 35206246, 35206247, 35206248, 35206249, 35206250, 35206251, 35206252, 35206253, 35206260, 35206261, 35206262, 35206263, 35206264, 35206265, 35206266, 35206274, 35206275, 35206276, 35206277, 35206278, 35206279, 35206280, 35206281, 35206282, 35206283, 35206288, 35206289, 35206290, 35206291, 35206292, 35206293, 35206294, 35206295, 35206296, 35206297, 35206298, 35206299, 35206300, 35206301, 35206302, 35206303, 35206304, 35206425,
#  35206427, 35206428, 35206430, 35206457, 35206458, 35206459, 35206461, 35206462, 35206466, 35206467, 35206470, 35206471, 35206472, 35206473, 35206474, 35206475, 35206476, 35206477, 35206478, 35206485, 35206486, 35206487, 35206488, 35206489, 35206490, 35206491, 35206492, 35206493, 35206494, 37200003, 37200004, 37200005, 37200006, 37200007, 37200008, 37200009, 44819421, 44819423, 44819424, 44819426, 44819427, 44819428, 44819429, 44819430, 44819431, 44819432, 44819433, 44819434, 44819435, 44819438, 44819439, 44819440, 44819444, 44819446, 44819448, 44819449, 44819455, 44819456, 44819458, 44819459, 44819464, 44819465, 44819469, 44819488, 44820594, 44820595, 44820596, 44820597, 44820598, 44820599, 44820600, 44820601, 44820602, 44820604, 44820605, 44820607, 44820608, 44820609, 44820610, 44820611, 44820612, 44820613, 44820614, 44820615, 44820616, 44820617, 44820619, 44820620, 44820621, 44820622, 44820624, 44820625, 44820626, 44820627, 44820629, 44820632, 44820636, 44820637, 44820641, 44820642,
#  44820643, 44820645, 44820646, 44820669, 44821721, 44821722, 44821723, 44821724, 44821725, 44821726, 44821727, 44821728, 44821729, 44821730, 44821731, 44821733, 44821734, 44821741, 44821742, 44821743, 44821745, 44821746, 44821747, 44821748, 44821749, 44821750, 44821756, 44821757, 44821758, 44821759, 44821760, 44821761, 44821762, 44821763, 44821775, 44821776, 44822865, 44822866, 44822867, 44822868, 44822870, 44822871, 44822872, 44822873, 44822874, 44822875, 44822876, 44822877, 44822878, 44822879, 44822880, 44822881, 44822882, 44822883, 44822884, 44822886, 44822887, 44822888, 44822889, 44822890, 44822891, 44822892, 44822893, 44822894, 44822895, 44822896, 44822897, 44822898, 44822899, 44822900, 44822901, 44822905, 44822907, 44822908, 44822910, 44822912, 44822923, 44822924, 44824020, 44824022, 44824023, 44824024, 44824025, 44824026, 44824027, 44824028, 44824029, 44824034, 44824038, 44824039, 44824041, 44824044, 44824045, 44824046, 44824047, 44824048, 44824051, 44824068, 44825188, 44825189,
#  44825190, 44825191, 44825192, 44825194, 44825195, 44825196, 44825197, 44825199, 44825200, 44825201, 44825202, 44825203, 44825204, 44825205, 44825206, 44825207, 44825208, 44825211, 44825212, 44825214, 44825215, 44825216, 44825217, 44825218, 44825219, 44825223, 44825224, 44825225, 44825229, 44825230, 44825231, 44825232, 44825233, 44825234, 44825236, 44825251, 44825252, 44826384, 44826386, 44826387, 44826388, 44826389, 44826390, 44826391, 44826392, 44826393, 44826394, 44826395, 44826396, 44826397, 44826398, 44826399, 44826400, 44826401, 44826402, 44826403, 44826404, 44826405, 44826406, 44826407, 44826408, 44826409, 44826413, 44826418, 44826419, 44826420, 44826422, 44826423, 44826424, 44826429, 44826430, 44826432, 44826433, 44826435, 44826449, 44826450, 44827542, 44827543, 44827544, 44827545, 44827546, 44827547, 44827549, 44827550, 44827551, 44827552, 44827553, 44827554, 44827555, 44827556, 44827558, 44827559, 44827560, 44827561, 44827562, 44827563, 44827569, 44827570, 44827571, 44827572,
#  44827573, 44827574, 44827577, 44827578, 44827579, 44827580, 44827583, 44827584, 44828724, 44828725, 44828726, 44828727, 44828728, 44828729, 44828730, 44828731, 44828733, 44828734, 44828735, 44828736, 44828737, 44828738, 44828739, 44828740, 44828741, 44828742, 44828743, 44828744, 44828748, 44828751, 44828753, 44828757, 44828758, 44828759, 44828773, 44828775, 44828776, 44829787, 44829788, 44829789, 44829791, 44829792, 44829793, 44829794, 44829795, 44829796, 44829797, 44829798, 44829800, 44829801, 44829802, 44829804, 44829805, 44829806, 44829807, 44829808, 44829809, 44829810, 44829812, 44829814, 44829815, 44829816, 44829817, 44829818, 44829825, 44829826, 44829828, 44829830, 44829833, 44829834, 44829837, 44829838, 44829839, 44829844, 44829845, 44829846, 44829847, 44829865, 44830967, 44830968, 44830969, 44830971, 44830972, 44830974, 44830975, 44830976, 44830977, 44830978, 44830979, 44830980, 44830981, 44830982, 44830983, 44830984, 44830985, 44830986, 44830987, 44830991, 44830992, 44830993,
#  44830994, 44830996, 44830997, 44830999, 44831000, 44831010, 44831013, 44831015, 44831016, 44831019, 44831033, 44831034, 44831036, 44832117, 44832118, 44832119, 44832120, 44832121, 44832122, 44832123, 44832124, 44832125, 44832126, 44832127, 44832128, 44832129, 44832131, 44832132, 44832137, 44832138, 44832139, 44832140, 44832141, 44832142, 44832143, 44832144, 44832145, 44832146, 44832147, 44832148, 44832149, 44832155, 44832156, 44832157, 44832158, 44832159, 44832160, 44832161, 44832162, 44832163, 44832165, 44832179, 44832180, 44832181, 44833280, 44833281, 44833282, 44833284, 44833285, 44833286, 44833287, 44833288, 44833289, 44833290, 44833291, 44833292, 44833293, 44833295, 44833296, 44833297, 44833298, 44833299, 44833300, 44833301, 44833303, 44833304, 44833305, 44833306, 44833311, 44833312, 44833313, 44833314, 44833315, 44833316, 44833317, 44833320, 44833321, 44833322, 44833323, 44833325, 44833326, 44833330, 44833331, 44833332, 44833334, 44833335, 44833338, 44833339, 44833343, 44833356,
#  44834474, 44834475, 44834476, 44834477, 44834480, 44834481, 44834482, 44834483, 44834486, 44834487, 44834488, 44834489, 44834490, 44834491, 44834493, 44834494, 44834495, 44834496, 44834498, 44834499, 44834504, 44834505, 44834506, 44834508, 44834510, 44834511, 44834512, 44834517, 44834518, 44834519, 44834521, 44834535, 44834536, 44834537, 44835660, 44835662, 44835664, 44835665, 44835667, 44835668, 44835669, 44835671, 44835672, 44835673, 44835674, 44835675, 44835676, 44835677, 44835678, 44835679, 44835680, 44835681, 44835687, 44835689, 44835690, 44835691, 44835692, 44835695, 44835696, 44835697, 44835699, 44835700, 44835703, 44835704, 44835707, 44835712, 44835713, 44835714, 44835734, 44835735, 44836817, 44836818, 44836819, 44836820, 44836821, 44836822, 44836823, 44836824, 44836825, 44836826, 44836827, 44836829, 44836830, 44836831, 44836834, 44836835, 44836836, 44836837, 44836839, 44836840, 44836842, 44836843, 44836844, 44836845, 44836846, 44836853, 44836854, 44836855, 44836856, 44836857,
#  44836858, 44836861, 44836862, 44836864, 44836866, 44836867, 44836868, 44836871, 44836898, 45532899, 45532904, 45532907, 45532908, 45532909, 45532910, 45532911, 45532912, 45532915, 45532919, 45532920, 45532923, 45532926, 45532927, 45532928, 45532932, 45532942, 45532943, 45532944, 45532945, 45532947, 45532948, 45532950, 45532951, 45532952, 45532953, 45532955, 45532958, 45532959, 45532960, 45532961, 45532963, 45532964, 45537790, 45537808, 45537809, 45537810, 45537811, 45537812, 45537813, 45537820, 45537821, 45537824, 45537828, 45537832, 45537833, 45537834, 45537846, 45537847, 45537848, 45537849, 45537850, 45537851, 45537852, 45537853, 45537854, 45537856, 45537857, 45537858, 45537859, 45537860, 45537861, 45537862, 45537863, 45537865, 45537866, 45537867, 45537868, 45537869, 45537870, 45537872, 45537874, 45537875, 45537877, 45537878, 45537883, 45537884, 45537886, 45542591, 45542593, 45542594, 45542595, 45542598, 45542599, 45542600, 45542601, 45542602, 45542603, 45542604, 45542605, 45542606,
#  45542607, 45542608, 45542609, 45542613, 45542615, 45542617, 45542619, 45542622, 45542623, 45542634, 45542635, 45542636, 45542637, 45542639, 45542640, 45542641, 45542643, 45542644, 45542645, 45542647, 45542648, 45542650, 45542652, 45542653, 45542655, 45542657, 45542658, 45542660, 45542662, 45542664, 45542665, 45542668, 45542672, 45542674, 45542676, 45542678, 45547486, 45547487, 45547488, 45547489, 45547490, 45547492, 45547495, 45547496, 45547497, 45547498, 45547499, 45547502, 45547503, 45547504, 45547514, 45547515, 45547518, 45547533, 45547534, 45547535, 45547536, 45547537, 45547538, 45547539, 45547540, 45547541, 45547542, 45547543, 45547544, 45547545, 45547548, 45547549, 45547550, 45547551, 45547552, 45547553, 45547554, 45547555, 45547557, 45547562, 45547566, 45547567, 45547568, 45547569, 45547570, 45547571, 45547573, 45552252, 45552253, 45552254, 45552255, 45552256, 45552259, 45552260, 45552262, 45552273, 45552274, 45552276, 45552278, 45552290, 45552291, 45552294, 45552295, 45552296,
#  45552297, 45552298, 45552300, 45552301, 45552302, 45552303, 45552304, 45552305, 45552306, 45552308, 45552311, 45552313, 45552314, 45552316, 45552320, 45552321, 45556973, 45556978, 45556979, 45556981, 45556982, 45556994, 45556995, 45556996, 45557001, 45557002, 45557003, 45557009, 45557010, 45557019, 45557020, 45557021, 45557022, 45557023, 45557024, 45557025, 45557026, 45557027, 45557028, 45557030, 45557031, 45557032, 45557033, 45557034, 45557035, 45557036, 45557038, 45557039, 45557040, 45557043, 45557045, 45557046, 45557052, 45557053, 45561783, 45561794, 45561795, 45561796, 45561797, 45561798, 45561799, 45561800, 45561801, 45561802, 45561803, 45561804, 45561807, 45561812, 45561813, 45561815, 45561817, 45561818, 45561819, 45561820, 45561823, 45561824, 45561825, 45561826, 45561837, 45561838, 45561839, 45561840, 45561841, 45561842, 45561843, 45561844, 45561845, 45561846, 45561847, 45561848, 45561849, 45561850, 45561851, 45561852, 45561853, 45561854, 45561855, 45561856, 45561859, 45561860,
#  45561861, 45561863, 45561864, 45561865, 45561866, 45561868, 45561871, 45561872, 45561873, 45561875, 45561876, 45561877, 45561879, 45561885, 45561890, 45566582, 45566584, 45566586, 45566588, 45566590, 45566592, 45566593, 45566594, 45566595, 45566596, 45566599, 45566600, 45566602, 45566603, 45566614, 45566615, 45566616, 45566621, 45566623, 45566625, 45566626, 45566627, 45566628, 45566629, 45566630, 45566632, 45566633, 45566634, 45566635, 45566636, 45566637, 45566638, 45566639, 45566641, 45566642, 45566643, 45566644, 45566645, 45566647, 45566649, 45566651, 45566658, 45566659, 45566661, 45571493, 45571494, 45571497, 45571499, 45571502, 45571503, 45571504, 45571505, 45571506, 45571507, 45571509, 45571513, 45571519, 45571520, 45571521, 45571522, 45571523, 45571525, 45571530, 45571531, 45571546, 45571547, 45571548, 45571550, 45571551, 45571552, 45571554, 45571557, 45571559, 45571561, 45571562, 45571563, 45571564, 45571565, 45571567, 45571568, 45571569, 45571573, 45571574, 45571577, 45571578,
#  45571583, 45571584, 45571585, 45571587, 45571588, 45576310, 45576313, 45576314, 45576316, 45576319, 45576323, 45576325, 45576327, 45576334, 45576336, 45576345, 45576346, 45576347, 45576348, 45576349, 45576350, 45576351, 45576352, 45576354, 45576355, 45576356, 45576357, 45576358, 45576360, 45576361, 45576362, 45576363, 45576364, 45576365, 45576366, 45576368, 45576369, 45576375, 45581204, 45581205, 45581206, 45581208, 45581209, 45581210, 45581211, 45581213, 45581216, 45581222, 45581223, 45581224, 45581225, 45581228, 45581229, 45581230, 45581231, 45581233, 45581234, 45581235, 45581236, 45581237, 45581238, 45581239, 45581240, 45581241, 45581243, 45581245, 45581246, 45581247, 45581248, 45581251, 45581253, 45581255, 45581257, 45581260, 45581269, 45581270, 45581272, 45581273, 45581274, 45581276, 45581277, 45585987, 45585992, 45585993, 45585994, 45585995, 45585996, 45585998, 45586000, 45586001, 45586002, 45586004, 45586005, 45586006, 45586010, 45586011, 45586012, 45586013, 45586014, 45586018,
#  45586019, 45586021, 45586022, 45586023, 45586026, 45586031, 45586032, 45586033, 45586034, 45586035, 45586036, 45586037, 45586038, 45586039, 45586040, 45586041, 45586042, 45586043, 45586045, 45586046, 45586047, 45586048, 45586049, 45586050, 45586052, 45586053, 45586054, 45586055, 45586057, 45586058, 45586060, 45586063, 45586064, 45586065, 45586066, 45586067, 45586068, 45586069, 45586070, 45586072, 45586077, 45586079, 45590884, 45590885, 45590888, 45590889, 45590890, 45590891, 45590892, 45590893, 45590894, 45590895, 45590896, 45590897, 45590898, 45590899, 45590901, 45590903, 45590904, 45590908, 45590909, 45590910, 45590913, 45590914, 45590916, 45590917, 45590928, 45590929, 45590931, 45590932, 45590933, 45590938, 45590940, 45590941, 45590942, 45590943, 45590944, 45590946, 45590948, 45590949, 45590951, 45590952, 45590954, 45590955, 45590956, 45590957, 45590959, 45590960, 45590963, 45590964, 45590966, 45590970, 45590972, 45590974, 45595653, 45595654, 45595655, 45595657, 45595659, 45595661,
#  45595663, 45595666, 45595667, 45595668, 45595677, 45595678, 45595679, 45595680, 45595681, 45595684, 45595694, 45595700, 45595701, 45595702, 45595703, 45595704, 45595705, 45595706, 45595707, 45595709, 45595710, 45595712, 45595713, 45595716, 45595717, 45595718, 45595719, 45595720, 45595721, 45595722, 45595723, 45595724, 45595728, 45595729, 45595730, 45595731, 45600488, 45600489, 45600490, 45600491, 45600494, 45600495, 45600497, 45600498, 45600499, 45600500, 45600501, 45600507, 45600508, 45600510, 45600511, 45600515, 45600517, 45600522, 45600523, 45600535, 45600536, 45600537, 45600538, 45600539, 45600540, 45600541, 45600542, 45600543, 45600544, 45600545, 45600547, 45600548, 45600550, 45600551, 45600553, 45600554, 45600555, 45600556, 45600558, 45600559, 45600560, 45600563, 45600567, 45600569, 45600570, 45600575, 45600576, 45600577, 45605267, 45605268, 45605269, 45605270, 45605271, 45605272, 45605276, 45605277, 45605284, 45605285, 45605286, 45605292, 45605293, 45605294, 45605298, 45605299,
#  45605301, 45605302, 45605304, 45605305, 45605306, 45605308, 45605309, 45605310, 45605311, 45605313, 45605314, 45605315, 45605316, 45605317, 45605318, 45605320, 45605321, 45605323, 45605324, 45605325, 45605326, 45605327, 45605329, 45605330, 45605331, 45605334, 45605335, 45605336, 45605339, 45605340, 45605341, 766339, 766341, 920116, 920117, 920118, 920119, 920120, 920121, 920122, 920123, 920124)       
#                         AND full_text LIKE '%_rank1]%'      ) a 
#                         ON (c.path LIKE CONCAT('%.', a.id, '.%') 
#                         OR c.path LIKE CONCAT('%.', a.id) 
#                         OR c.path LIKE CONCAT(a.id, '.%') 
#                         OR c.path = a.id) 
#                 WHERE
#                     is_standard = 0 
#                     AND is_selectable = 1)
#             )  
#             AND (
#                 c_occurrence.PERSON_ID IN (SELECT
#                     distinct person_id  
#                 FROM
#                     `cb_search_person` cb_search_person  
#                 WHERE
#                     cb_search_person.person_id IN (SELECT
#                         person_id 
#                     FROM
#                         `cb_search_person` p 
#                     WHERE
#                         has_ehr_data = 1 ) 
#                     AND cb_search_person.person_id IN (SELECT
#                         person_id 
#                     FROM
#                         `cb_search_person` p 
#                     WHERE
#                         has_whole_genome_variant = 1 ) 
#                     AND cb_search_person.person_id IN (SELECT
#                         criteria.person_id 
#                     FROM
#                         (SELECT
#                             DISTINCT person_id, entry_date, concept_id 
#                         FROM
#                             `cb_search_all_events` 
#                         WHERE
#                             (concept_id IN(SELECT
#                                 DISTINCT c.concept_id 
#                             FROM
#                                 `cb_criteria` c 
#                             JOIN
#                                 (SELECT
#                                     CAST(cr.id as string) AS id       
#                                 FROM
#                                     `cb_criteria` cr       
#                                 WHERE
#                                     concept_id IN (35206491, 44836821, 44828755, 35206457, 44820669, 1567641, 44824027, 44822872, 1567492, 44828775, 1567651, 44827551, 35206249, 44826387, 1567478, 44827552, 1567462, 44831033, 1567729, 44827586, 44834505, 1567461, 44827555, 44827554, 44833291, 44832117, 35206466, 1567586, 44826401, 44825211, 44828741, 35206458, 1567584, 1567674, 44829811, 1567515, 1567721, 44826398, 44833294, 1567537, 44832129, 44820621, 44825192, 1567530, 35206139, 44819440, 44834496, 44828731, 35206153, 35206140, 1567581, 35206252, 44829795, 1567582, 44825196, 44825200, 44831017, 45581270, 1567597, 44829838, 44835663, 44835667, 1567722, 44833313, 44832131, 44826394, 44819488, 45595728, 44825199, 44832180, 44819422, 1567469, 44836825, 44820604, 44829814, 44830987, 44826392, 1567659, 1567668, 35206459, 1567479, 44828776, 44835690, 35206131, 35206266, 35206461, 1567502, 44836826, 44829802, 44836845, 44830972, 1567473, 44834480, 35206248, 44826449, 44827579, 44825214,
#  44830991, 1567574, 44834482, 1567568, 44835662, 45605336, 35206185, 44835735, 44820605, 35206247, 44826450, 35206137, 44822889, 45581269, 1567483, 44827547, 45547562, 1567529, 1567680, 44830997, 44829789, 1567735, 44826391, 44833300, 44824034, 1567494, 44833283, 44835673, 44828750, 44821756, 44831036, 44830967, 35206242, 44824068, 44826409, 35206183, 35206133, 35206260, 1567675, 44836830, 44820600, 1567501, 1567466, 44825195, 35206492, 1567463, 35206304, 44821775, 35206132, 1567471, 44833301, 1567746, 44832181, 44822887, 35206134, 44826420, 44836846, 44836837, 44828729, 35206467, 35206251, 44833282, 44825252, 44829788, 1567699, 1567596, 44833287, 1567470, 44836831, 35206138, 35206056, 44833292, 1567534, 35206250, 1567472, 44830971, 44819425, 1567652, 44819434, 44829800, 44824023, 1567740, 1567476, 1567689, 44834486, 44819421, 44835668, 44828733, 1567573, 44825251, 44833356, 1567583, 44830968, 35206489, 35206253, 44835665, 44835712, 44832148, 1567633, 1567474, 44827556, 35206101,
#  44836833, 44835677, 44833293, 44834535, 44827545, 1567468, 44826393, 44833306, 44833334, 1567673, 1567485, 35206130, 44832119, 44833284, 44825194, 1567705, 44827544, 44821758, 44822893, 44825208, 44827563, 35206135, 44820628, 1567484, 44824020, 44822870, 1567598, 44828744, 44826399, 35206493, 35206080, 44820616, 44825191, 1567464, 44836844, 44820617, 35206494, 35206490, 1567465, 35206136, 44829815, 44825232, 1567603, 35206141, 1567528, 44822886, 44826389, 35206462, 44835671, 1567475, 1567481, 44835693, 44827548, 44827546, 44824024, 1567566, 1567486, 44832122, 44826386, 44826413, 1567565, 1567533, 44832179, 44830969)       
#                                     AND full_text LIKE '%_rank1]%'      ) a 
#                                     ON (c.path LIKE CONCAT('%.', a.id, '.%') 
#                                     OR c.path LIKE CONCAT('%.', a.id) 
#                                     OR c.path LIKE CONCAT(a.id, '.%') 
#                                     OR c.path = a.id) 
#                             WHERE
#                                 is_standard = 0 
#                                 AND is_selectable = 1) 
#                             AND is_standard = 0 )) criteria )  
#                 UNION
#                 DISTINCT SELECT
#                     distinct person_id  
#                 FROM
#                     `cb_search_person` cb_search_person  
#                 WHERE
#                     cb_search_person.person_id IN (SELECT
#                         person_id 
#                     FROM
#                         `cb_search_person` p 
#                     WHERE
#                         has_ehr_data = 1 ) 
#                     AND cb_search_person.person_id IN (SELECT
#                         person_id 
#                     FROM
#                         `cb_search_person` p 
#                     WHERE
#                         has_whole_genome_variant = 1 ) 
#                     AND cb_search_person.person_id NOT IN (SELECT
#                         criteria.person_id 
#                     FROM
#                         (SELECT
#                             DISTINCT person_id, entry_date, concept_id 
#                         FROM
#                             `cb_search_all_events` 
#                         WHERE
#                             (concept_id IN(SELECT
#                                 DISTINCT c.concept_id 
#                             FROM
#                                 `cb_criteria` c 
#                             JOIN
#                                 (SELECT
#                                     CAST(cr.id as string) AS id       
#                                 FROM
#                                     `cb_criteria` cr       
#                                 WHERE
#                                     concept_id IN (35206491, 44836821, 44828755, 35206457, 44820669, 1567641, 44824027, 44822872, 1567492, 44828775, 1567651, 44827551, 35206249, 44826387, 1567478, 44827552, 1567462, 44831033, 1567729, 44827586, 44834505, 1567461, 44827555, 44827554, 44833291, 44832117, 35206466, 1567586, 44826401, 44825211, 44828741, 35206458, 1567584, 1567674, 44829811, 1567515, 1567721, 44826398, 44833294, 1567537, 44832129, 44820621, 44825192, 1567530, 35206139, 44819440, 44834496, 44828731, 35206153, 35206140, 1567581, 35206252, 44829795, 1567582, 44825196, 44825200, 44831017, 45581270, 1567597, 44829838, 44835663, 44835667, 1567722, 44833313, 44832131, 44826394, 44819488, 45595728, 44825199, 44832180, 44819422, 1567469, 44836825, 44820604, 44829814, 44830987, 44826392, 1567659, 1567668, 35206459, 1567479, 44828776, 44835690, 35206131, 35206266, 35206461, 1567502, 44836826, 44829802, 44836845, 44830972, 1567473, 44834480, 35206248, 44826449, 44827579, 44825214,
#  44830991, 1567574, 44834482, 1567568, 44835662, 45605336, 35206185, 44835735, 44820605, 35206247, 44826450, 35206137, 44822889, 45581269, 1567483, 44827547, 45547562, 1567529, 1567680, 44830997, 44829789, 1567735, 44826391, 44833300, 44824034, 1567494, 44833283, 44835673, 44828750, 44821756, 44831036, 44830967, 35206242, 44824068, 44826409, 35206183, 35206133, 35206260, 1567675, 44836830, 44820600, 1567501, 1567466, 44825195, 35206492, 1567463, 35206304, 44821775, 35206132, 1567471, 44833301, 1567746, 44832181, 44822887, 35206134, 44826420, 44836846, 44836837, 44828729, 35206467, 35206251, 44833282, 44825252, 44829788, 1567699, 1567596, 44833287, 1567470, 44836831, 35206138, 35206056, 44833292, 1567534, 35206250, 1567472, 44830971, 44819425, 1567652, 44819434, 44829800, 44824023, 1567740, 1567476, 1567689, 44834486, 44819421, 44835668, 44828733, 1567573, 44825251, 44833356, 1567583, 44830968, 35206489, 35206253, 44835665, 44835712, 44832148, 1567633, 1567474, 44827556, 35206101,
#  44836833, 44835677, 44833293, 44834535, 44827545, 1567468, 44826393, 44833306, 44833334, 1567673, 1567485, 35206130, 44832119, 44833284, 44825194, 1567705, 44827544, 44821758, 44822893, 44825208, 44827563, 35206135, 44820628, 1567484, 44824020, 44822870, 1567598, 44828744, 44826399, 35206493, 35206080, 44820616, 44825191, 1567464, 44836844, 44820617, 35206494, 35206490, 1567465, 35206136, 44829815, 44825232, 1567603, 35206141, 1567528, 44822886, 44826389, 35206462, 44835671, 1567475, 1567481, 44835693, 44827548, 44827546, 44824024, 1567566, 1567486, 44832122, 44826386, 44826413, 1567565, 1567533, 44832179, 44830969)       
#                                     AND full_text LIKE '%_rank1]%'      ) a 
#                                     ON (c.path LIKE CONCAT('%.', a.id, '.%') 
#                                     OR c.path LIKE CONCAT('%.', a.id) 
#                                     OR c.path LIKE CONCAT(a.id, '.%') 
#                                     OR c.path = a.id) 
#                             WHERE
#                                 is_standard = 0 
#                                 AND is_selectable = 1) 
#                             AND is_standard = 0 )) criteria ) 
#                     AND cb_search_person.person_id NOT IN (SELECT
#                         criteria.person_id 
#                     FROM
#                         (SELECT
#                             DISTINCT person_id, entry_date, concept_id 
#                         FROM
#                             `cb_search_all_events` 
#                         WHERE
#                             (concept_id IN (43021268, 4324321, 46273375, 46270536, 4201829, 46273407, 35610791, 46273418, 46270077, 762250, 4187203, 4178769, 43021272, 4333345, 4176919, 4325868, 4212564, 4190635, 4324190, 4327107, 4179242, 4190634, 4212563, 46273480, 4187206, 4180749, 44782983, 4181024, 37016185, 4327885, 4325851, 4190633, 46273372, 4324189, 4324191, 4178782, 40483717, 4179084, 4187205, 4333465, 43021269, 4177063, 4216132, 4190632, 4180131, 4180113, 4197758, 4144289, 43021271, 4212565, 37016142, 45763741, 37018569, 4179069, 46274037) 
#                             AND is_standard = 1 )) criteria ) )
#             )
#         ) c_occurrence 
#     LEFT JOIN
#         `concept` c_standard_concept 
#             ON c_occurrence.condition_concept_id = c_standard_concept.concept_id 
#     LEFT JOIN
#         `concept` c_type 
#             ON c_occurrence.condition_type_concept_id = c_type.concept_id 
#     LEFT JOIN
#         `visit_occurrence` v 
#             ON c_occurrence.visit_occurrence_id = v.visit_occurrence_id 
#     LEFT JOIN
#         `concept` visit 
#             ON v.visit_concept_id = visit.concept_id 
#     LEFT JOIN
#         `concept` c_source_concept 
#             ON c_occurrence.condition_source_concept_id = c_source_concept.concept_id 
#     LEFT JOIN
#         `concept` c_status 
#             ON c_occurrence.condition_status_concept_id = c_status.concept_id", sep="")

# # Formulate a Cloud Storage destination path for the data exported from BigQuery.
# # NOTE: By default data exported multiple times on the same day will overwrite older copies.
# #       But data exported on a different days will write to a new location so that historical
# #       copies can be kept as the dataset definition is changed.
# condition_79764376_path <- file.path(
#   Sys.getenv("WORKSPACE_BUCKET"),
#   "AoU_cancer",
#   "phenotypes",
#   "condition_cancer_data.csv")
# message(str_glue('The data will be written to {condition_79764376_path}. Use this path when reading ',
#                  'the data into your notebooks in the future.'))
             
# # Perform the query and export the dataset to Cloud Storage as CSV files.
# # NOTE: You only need to run `bq_table_save` once. After that, you can
# #       just read data from the CSVs in Cloud Storage.
# bq_table_save(
#   bq_dataset_query(Sys.getenv("WORKSPACE_CDR"), dataset_79764376_condition_sql, billing = Sys.getenv("GOOGLE_PROJECT")),
#   condition_79764376_path,
#   destination_format = "CSV")

# # This query represents dataset "All srWGS cancer data" for domain "survey" and was generated for All of Us Controlled Tier Dataset v8
# dataset_79764376_survey_sql <- paste("
#     SELECT
#         answer.person_id,
#         answer.survey_datetime  
#     FROM
#         `ds_survey` answer   
#     WHERE
#         (
#             question_concept_id IN (SELECT
#                 DISTINCT concept_id                         
#             FROM
#                 `cb_criteria` c                         
#             JOIN
#                 (SELECT
#                     CAST(cr.id as string) AS id                               
#                 FROM
#                     `cb_criteria` cr                               
#                 WHERE
#                     concept_id IN (1586134)                               
#                     AND domain_id = 'SURVEY') a 
#                     ON (c.path like CONCAT('%', a.id, '.%'))                         
#             WHERE
#                 domain_id = 'SURVEY'                         
#                 AND type = 'PPI'                         
#                 AND subtype = 'QUESTION')
#         )  
#         AND (
#             answer.PERSON_ID IN (SELECT
#                 distinct person_id  
#             FROM
#                 `cb_search_person` cb_search_person  
#             WHERE
#                 cb_search_person.person_id IN (SELECT
#                     person_id 
#                 FROM
#                     `cb_search_person` p 
#                 WHERE
#                     has_ehr_data = 1 ) 
#                 AND cb_search_person.person_id IN (SELECT
#                     person_id 
#                 FROM
#                     `cb_search_person` p 
#                 WHERE
#                     has_whole_genome_variant = 1 ) 
#                 AND cb_search_person.person_id IN (SELECT
#                     criteria.person_id 
#                 FROM
#                     (SELECT
#                         DISTINCT person_id, entry_date, concept_id 
#                     FROM
#                         `cb_search_all_events` 
#                     WHERE
#                         (concept_id IN(SELECT
#                             DISTINCT c.concept_id 
#                         FROM
#                             `cb_criteria` c 
#                         JOIN
#                             (SELECT
#                                 CAST(cr.id as string) AS id       
#                             FROM
#                                 `cb_criteria` cr       
#                             WHERE
#                                 concept_id IN (35206491, 44836821, 44828755, 35206457, 44820669, 1567641, 44824027, 44822872, 1567492, 44828775, 1567651, 44827551, 35206249, 44826387, 1567478, 44827552, 1567462, 44831033, 1567729, 44827586, 44834505, 1567461, 44827555, 44827554, 44833291, 44832117, 35206466, 1567586, 44826401, 44825211, 44828741, 35206458, 1567584, 1567674, 44829811, 1567515, 1567721, 44826398, 44833294, 1567537, 44832129, 44820621, 44825192, 1567530, 35206139, 44819440, 44834496, 44828731, 35206153, 35206140, 1567581, 35206252, 44829795, 1567582, 44825196, 44825200, 44831017, 45581270, 1567597, 44829838, 44835663, 44835667, 1567722, 44833313, 44832131, 44826394, 44819488, 45595728, 44825199, 44832180, 44819422, 1567469, 44836825, 44820604, 44829814, 44830987, 44826392, 1567659, 1567668, 35206459, 1567479, 44828776, 44835690, 35206131, 35206266, 35206461, 1567502, 44836826, 44829802, 44836845, 44830972, 1567473, 44834480, 35206248, 44826449, 44827579, 44825214,
#  44830991, 1567574, 44834482, 1567568, 44835662, 45605336, 35206185, 44835735, 44820605, 35206247, 44826450, 35206137, 44822889, 45581269, 1567483, 44827547, 45547562, 1567529, 1567680, 44830997, 44829789, 1567735, 44826391, 44833300, 44824034, 1567494, 44833283, 44835673, 44828750, 44821756, 44831036, 44830967, 35206242, 44824068, 44826409, 35206183, 35206133, 35206260, 1567675, 44836830, 44820600, 1567501, 1567466, 44825195, 35206492, 1567463, 35206304, 44821775, 35206132, 1567471, 44833301, 1567746, 44832181, 44822887, 35206134, 44826420, 44836846, 44836837, 44828729, 35206467, 35206251, 44833282, 44825252, 44829788, 1567699, 1567596, 44833287, 1567470, 44836831, 35206138, 35206056, 44833292, 1567534, 35206250, 1567472, 44830971, 44819425, 1567652, 44819434, 44829800, 44824023, 1567740, 1567476, 1567689, 44834486, 44819421, 44835668, 44828733, 1567573, 44825251, 44833356, 1567583, 44830968, 35206489, 35206253, 44835665, 44835712, 44832148, 1567633, 1567474, 44827556, 35206101,
#  44836833, 44835677, 44833293, 44834535, 44827545, 1567468, 44826393, 44833306, 44833334, 1567673, 1567485, 35206130, 44832119, 44833284, 44825194, 1567705, 44827544, 44821758, 44822893, 44825208, 44827563, 35206135, 44820628, 1567484, 44824020, 44822870, 1567598, 44828744, 44826399, 35206493, 35206080, 44820616, 44825191, 1567464, 44836844, 44820617, 35206494, 35206490, 1567465, 35206136, 44829815, 44825232, 1567603, 35206141, 1567528, 44822886, 44826389, 35206462, 44835671, 1567475, 1567481, 44835693, 44827548, 44827546, 44824024, 1567566, 1567486, 44832122, 44826386, 44826413, 1567565, 1567533, 44832179, 44830969)       
#                                 AND full_text LIKE '%_rank1]%'      ) a 
#                                 ON (c.path LIKE CONCAT('%.', a.id, '.%') 
#                                 OR c.path LIKE CONCAT('%.', a.id) 
#                                 OR c.path LIKE CONCAT(a.id, '.%') 
#                                 OR c.path = a.id) 
#                         WHERE
#                             is_standard = 0 
#                             AND is_selectable = 1) 
#                         AND is_standard = 0 )) criteria )  
#             UNION
#             DISTINCT SELECT
#                 distinct person_id  
#             FROM
#                 `cb_search_person` cb_search_person  
#             WHERE
#                 cb_search_person.person_id IN (SELECT
#                     person_id 
#                 FROM
#                     `cb_search_person` p 
#                 WHERE
#                     has_ehr_data = 1 ) 
#                 AND cb_search_person.person_id IN (SELECT
#                     person_id 
#                 FROM
#                     `cb_search_person` p 
#                 WHERE
#                     has_whole_genome_variant = 1 ) 
#                 AND cb_search_person.person_id NOT IN (SELECT
#                     criteria.person_id 
#                 FROM
#                     (SELECT
#                         DISTINCT person_id, entry_date, concept_id 
#                     FROM
#                         `cb_search_all_events` 
#                     WHERE
#                         (concept_id IN(SELECT
#                             DISTINCT c.concept_id 
#                         FROM
#                             `cb_criteria` c 
#                         JOIN
#                             (SELECT
#                                 CAST(cr.id as string) AS id       
#                             FROM
#                                 `cb_criteria` cr       
#                             WHERE
#                                 concept_id IN (35206491, 44836821, 44828755, 35206457, 44820669, 1567641, 44824027, 44822872, 1567492, 44828775, 1567651, 44827551, 35206249, 44826387, 1567478, 44827552, 1567462, 44831033, 1567729, 44827586, 44834505, 1567461, 44827555, 44827554, 44833291, 44832117, 35206466, 1567586, 44826401, 44825211, 44828741, 35206458, 1567584, 1567674, 44829811, 1567515, 1567721, 44826398, 44833294, 1567537, 44832129, 44820621, 44825192, 1567530, 35206139, 44819440, 44834496, 44828731, 35206153, 35206140, 1567581, 35206252, 44829795, 1567582, 44825196, 44825200, 44831017, 45581270, 1567597, 44829838, 44835663, 44835667, 1567722, 44833313, 44832131, 44826394, 44819488, 45595728, 44825199, 44832180, 44819422, 1567469, 44836825, 44820604, 44829814, 44830987, 44826392, 1567659, 1567668, 35206459, 1567479, 44828776, 44835690, 35206131, 35206266, 35206461, 1567502, 44836826, 44829802, 44836845, 44830972, 1567473, 44834480, 35206248, 44826449, 44827579, 44825214,
#  44830991, 1567574, 44834482, 1567568, 44835662, 45605336, 35206185, 44835735, 44820605, 35206247, 44826450, 35206137, 44822889, 45581269, 1567483, 44827547, 45547562, 1567529, 1567680, 44830997, 44829789, 1567735, 44826391, 44833300, 44824034, 1567494, 44833283, 44835673, 44828750, 44821756, 44831036, 44830967, 35206242, 44824068, 44826409, 35206183, 35206133, 35206260, 1567675, 44836830, 44820600, 1567501, 1567466, 44825195, 35206492, 1567463, 35206304, 44821775, 35206132, 1567471, 44833301, 1567746, 44832181, 44822887, 35206134, 44826420, 44836846, 44836837, 44828729, 35206467, 35206251, 44833282, 44825252, 44829788, 1567699, 1567596, 44833287, 1567470, 44836831, 35206138, 35206056, 44833292, 1567534, 35206250, 1567472, 44830971, 44819425, 1567652, 44819434, 44829800, 44824023, 1567740, 1567476, 1567689, 44834486, 44819421, 44835668, 44828733, 1567573, 44825251, 44833356, 1567583, 44830968, 35206489, 35206253, 44835665, 44835712, 44832148, 1567633, 1567474, 44827556, 35206101,
#  44836833, 44835677, 44833293, 44834535, 44827545, 1567468, 44826393, 44833306, 44833334, 1567673, 1567485, 35206130, 44832119, 44833284, 44825194, 1567705, 44827544, 44821758, 44822893, 44825208, 44827563, 35206135, 44820628, 1567484, 44824020, 44822870, 1567598, 44828744, 44826399, 35206493, 35206080, 44820616, 44825191, 1567464, 44836844, 44820617, 35206494, 35206490, 1567465, 35206136, 44829815, 44825232, 1567603, 35206141, 1567528, 44822886, 44826389, 35206462, 44835671, 1567475, 1567481, 44835693, 44827548, 44827546, 44824024, 1567566, 1567486, 44832122, 44826386, 44826413, 1567565, 1567533, 44832179, 44830969)       
#                                 AND full_text LIKE '%_rank1]%'      ) a 
#                                 ON (c.path LIKE CONCAT('%.', a.id, '.%') 
#                                 OR c.path LIKE CONCAT('%.', a.id) 
#                                 OR c.path LIKE CONCAT(a.id, '.%') 
#                                 OR c.path = a.id) 
#                         WHERE
#                             is_standard = 0 
#                             AND is_selectable = 1) 
#                         AND is_standard = 0 )) criteria ) 
#                 AND cb_search_person.person_id NOT IN (SELECT
#                     criteria.person_id 
#                 FROM
#                     (SELECT
#                         DISTINCT person_id, entry_date, concept_id 
#                     FROM
#                         `cb_search_all_events` 
#                     WHERE
#                         (concept_id IN (43021268, 4324321, 46273375, 46270536, 4201829, 46273407, 35610791, 46273418, 46270077, 762250, 4187203, 4178769, 43021272, 4333345, 4176919, 4325868, 4212564, 4190635, 4324190, 4327107, 4179242, 4190634, 4212563, 46273480, 4187206, 4180749, 44782983, 4181024, 37016185, 4327885, 4325851, 4190633, 46273372, 4324189, 4324191, 4178782, 40483717, 4179084, 4187205, 4333465, 43021269, 4177063, 4216132, 4190632, 4180131, 4180113, 4197758, 4144289, 43021271, 4212565, 37016142, 45763741, 37018569, 4179069, 46274037) 
#                         AND is_standard = 1 )) criteria ) )
#         )", sep="")

# # Formulate a Cloud Storage destination path for the data exported from BigQuery.
# # NOTE: By default data exported multiple times on the same day will overwrite older copies.
# #       But data exported on a different days will write to a new location so that historical
# #       copies can be kept as the dataset definition is changed.
# survey_79764376_path <- file.path(
#   Sys.getenv("WORKSPACE_BUCKET"),
#   "AoU_cancer",
#   "phenotypes",
#   "survey_cancer_data.csv")
# message(str_glue('The data will be written to {survey_79764376_path}. Use this path when reading ',
#                  'the data into your notebooks in the future.'))

# # Perform the query and export the dataset to Cloud Storage as CSV files.
# # NOTE: You only need to run `bq_table_save` once. After that, you can
# #       just read data from the CSVs in Cloud Storage.
# bq_table_save(
#   bq_dataset_query(Sys.getenv("WORKSPACE_CDR"), dataset_79764376_survey_sql, billing = Sys.getenv("GOOGLE_PROJECT")),
#   survey_79764376_path,
#   destination_format = "CSV")

### Load the data sets and check variables

In [ ]:
# Read the data directly from Cloud Storage into memory.
# NOTE: Alternatively you can `gsutil -m cp {person_02449410_path}` to copy these files
#       to the Jupyter disk.
read_bq_export_from_workspace_bucket <- function(export_path) {
  col_types <- cols(gender = col_character(), race = col_character(), ethnicity = col_character(), sex_at_birth = col_character())
  bind_rows(
    map(system2('gsutil', args = c('ls', export_path), stdout = TRUE, stderr = TRUE),
        function(csv) {
          message(str_glue('Loading {csv}.'))
          chunk <- read_csv(pipe(str_glue('gsutil cat {csv}')), col_types = col_types, show_col_types = FALSE)
          if (is.null(col_types)) {
            col_types <- spec(chunk)
          }
          chunk
        }))
}

demographics_path = "gs://fc-secure-f2ecfa45-d7db-4232-929f-8cf417be1eb0/AoU_cancer/phenotypes/person_cancer_data.csv"
condition_path = "gs://fc-secure-f2ecfa45-d7db-4232-929f-8cf417be1eb0/AoU_cancer/phenotypes/condition_cancer_data.csv"
survey_path = "gs://fc-secure-f2ecfa45-d7db-4232-929f-8cf417be1eb0/AoU_cancer/phenotypes/survey_cancer_data.csv"

demographics <- read_bq_export_from_workspace_bucket(demographics_path)
condition <- read_bq_export_from_workspace_bucket(condition_path)
survey <- read_bq_export_from_workspace_bucket(survey_path)

## Create a condition data set with the first instance of each ICD code for each individual

### Create a numeric date variable 

In [ ]:
library(data.table)
data_condition = condition %>% mutate(diagnosis_date = as.IDate(condition_start_datetime))

### Extract and store the first chronological instance of each ICD code for each individual

In [ ]:
identify_first_code_instance <- function(X) {
  earliest_instance <- X %>%
    group_by(person_id, source_concept_code) %>%
    filter(diagnosis_date == min(diagnosis_date)) %>%
    ungroup()
  
  return(earliest_instance)
}

data_first_condition = identify_first_code_instance(data_condition)

## Classify cancer sites based on ICD 9 and ICD 10 codes

### Create a list of ICD code to cancer site conversions 

In [ ]:
lip = c("140",paste0("140.",0:9),"C00",paste0("C00.",0:9))
tongue = c("141",paste0("141.",0:9),"C01",paste0("C01.",0:9),"C02",paste0("C02.",0:9))
is_tongue = c("D00.07")
gum = c("143",paste0("143.",0:9),"C03",paste0("C03.",0:9))
mouth_floor = c("144",paste0("144.",0:9),"C04",paste0("C04.",0:9))
is_mouth_floor = c("D00.06")
palate = c(paste0("145.",2:5),"C05",paste0("C05.",0:9))
is_palate = c("D00.04","D00.05")
other_mouth = c("145.9","C06",paste0("C06.",0:9))
is_other_mouth = c("D00.00")
parotid_gland = c("142.0","C07",paste0("C07.",0:9))
salivary_glands = c(paste0("142.",1:9),"C08",paste0("C08.",0:9))
tonsil = c(paste0("146.",0:2),"C09",paste0("C09.",0:9))
oropharynx = c(paste0("146.",3:9),"C10",paste0("C10.",0:9))
nasopharynx = c("147",paste0("147.",0:9),"C11",paste0("C11.",0:9))
pyriform_sinus = c("148.1","C12",paste0("C12.",0:9))
hypopharynx = c("148.0",paste0("148.",2:9),"C13",paste0("C13.",0:9))
other_head_and_neck = c("149",paste0("149.",0:9),"C14",paste0("C14.",0:9))
esophagus = c("150",paste0("150.",0:9),"C15",paste0("C15.",0:9))
is_esophagus = c("D00.1","D00.10","230.1")
stomach = c("151",paste0("151.",0:9),"C16",paste0("C16.",0:9))
is_stomach = c("D00.2","D00.20","230.2")
small_intestine = c("152",paste0("152.",0:9),"C17",paste0("C17.",0:9))
appendix = c("153.5","C18.1")
colon = c("153",paste0("153.",0:4),paste0("153.",6:9),"C18.0",paste0("C18.",2:9))
is_colon = c("D01.0","230.3")
rectosigmoid_junction = c("154.0","C19",paste0("C19.",0:9))
rectum = c("154.1","C20",paste0("C20",0:9))
is_rectum = c("D01.2","230.4")
anus = c("154.2","154.3","C21",paste0("C21.",0:9))
is_anus = c("D01.3","230.5","230.6")
liver = c("155",paste0("155.",0:2),"C22",paste0("C22.",0:9))
gallbladder = c("156",paste0("156.",0:9),"C23",paste0("C23.",0:9))
pancreas = c("157",paste0("157.",0:9),"C25",paste0("C25.",0:9))
nasal_ear = c(paste0("160.",0:1),"C30",paste0("C30.",0:9))
accessory_sinuses = c(paste0("160.",2:9),"C31",paste0("C31.",0:9))
larynx = c("161",paste0("161.",0:9),"C32",paste0("C32.",0:9))
is_larynx = c("D02.0","231.0")
trachea = c("162.0","C33",paste0("C33.",0:9))
is_trachea = c("D02.1","231.1")
lung = c(paste0("162.",1:9),"C34",paste0("C34.",0:9))
is_lung = c("D02.2","231.2")
thymus = c("164.0","C37",paste0("C37.",0:9))
heart_other = c("163",paste0("163.",0:9),paste0("164.",1:9),"C38",paste0("C38.",0:9))
bone = c("170",paste0("170.",0:9),"C40","C41",paste0("C40.",0:9),paste0("C41.",0:9))
melanoma = c("172",paste0("172.",0:9),"C43",paste0("C43.",0:9))
is_melanoma = c("D03",paste0("D03.",0:9))
non_melanoma_skin = c("173",paste0("173.",0:9),"C44",paste0("C44.",0:9))
is_non_melanoma_skin = c("D04",paste0("D04.",0:9),"232",paste0("232.",0:9))
mesothelioma = c("C45",paste0("C45.",0:9))
Kaposi_sarcoma = c("176",paste0("176.",0:9),"C46",paste0("C46.",0:9))
peripheral_nervous = c("C47",paste0("C47.",0:9))
retroperitoneum_peritoneum = c("158",paste0("158.",0:9),"C48",paste0("C48.",0:9))
other_soft = c("171",paste0("171.",0:9),"C49",paste0("C49.",0:9),"C49.A")
breast = c("174","175",paste0("174.",0:9),paste0("175.",0:9),"C50",paste0("C50.",0:9))
is_breast = c("D05",paste0("D05.",0:9),"233.0")
vulva = c(paste0("184.",1:4),"C51",paste0("C51.",0:9))
is_vulva = c("D07.1","233.3")
vagina = c("184.0","C52",paste0("C52.",0:9))
is_vagina = c("D07.2","233.31")
cervix = c("180",paste0("180.",0:9),"C53",paste0("C53.",0:9))
is_cervix = c("D06",paste0("D06.",0:9),"233.1")
other_uterus = c("182",paste0("182.",0:9),"C55","C54.0",paste0("C55.",0:9),paste0("C54.",2:9))
endometrium = c("C54.1")
is_endometrium = c("D07.0")
ovary = c("183.0","C56",paste0("C56.",0:9))
placenta = c("181",paste0("181.",0:9),"C58",paste0("C58.",0:9))
penis = c(paste0("187.",1:4),"C60",paste0("C60.",0:9))
is_penis = c("D07.4","233.5")
prostate = c("185",paste0("185.",0:9),"C61",paste0("C61.",0:9))
is_prostate = c("D07.5","233.4")
testis = c("186",paste0("186.",0:9),"C62",paste0("C62.",0:9))
kidney = c("189.0","C64",paste0("C64.",0:9))
renal_pelvis = c("189.1","C65",paste0("C65.",0:9))
ureter = c("189.2","C66",paste0("C66.",0:9))
bladder = c("188",paste0("188.",0:9),"C67",paste0("C67.",0:9))
is_bladder = c("D09.0","233.7")
eye = c("190",paste0("190.",0:9),"C69",paste0("C69.",0:9))
is_eye = c("D09.2","234.0")
meninges = c("192.1","C70",paste0("C70.",0:9))
brain = c("191",paste0("191.",0:9),"C71",paste0("C71.",0:9))
other_central_nervous = c("192","192.0",paste0("192.",2:9),"C72",paste0("C72.",0:9))
thyroid_gland = c("193",paste0("193.",0:9),"C73",paste0("C73.",0:9))
adrenal_gland = c("194.0","C74",paste0("C74.",0:9))
Hodgkin_lymphoma = c("201",paste0("201.",0:9),"C81",paste0("C81.",0:9))
follicular_lymphoma = c("202.0","C82",paste0("C82.",0:9))
non_follicular_lymphoma = c("200",paste0("200.",0:5),paste0("200.",7:9),"C83",paste0("C83.",0:9))
T_NK_lymphoma = c("200.6","202.1","202.2","202.7","C84",paste0("C84.",0:9),"C84.A","C84.Z")
other_non_Hodgkin_lymphoma = c("C85",paste0("C85.",0:9))
other_T_NK_lymphoma = c("C86",paste0("C86.",0:9))
immunoproliferative = c("203.8","C88",paste0("C88.",0:9))
multiple_myeloma = c("203",paste0("203.",0:7),"203.9","C90",paste0("C90.",0:9))
lymphoid_leukemia = c("204",paste0("204.",0:9),"C91",paste0("C91.",0:9),"C91.Z","C91.A")
myeloid_leukemia = c("205",paste0("205.",0:9),"C92",paste0("C92.",0:9))
monocytic_leukemia = c("206",paste0("206.",0:9),"C93",paste0("C93.",0:9),"C93.Z")
other_leukemia = c("207",paste0("207.",0:9),"C94",paste0("C94.",0:9))

### Create a cancer site and invasive indicator variable based on the ICD 9 and ICD 10 codes 

In [ ]:
data_first_condition = data_first_condition %>%
mutate(cancer_site = case_when(substr(source_concept_code, 1, 5) %in% lip ~ "lip",
                               substr(source_concept_code, 1, 5) %in% c(tongue, is_tongue) ~ "tongue",
                               substr(source_concept_code, 1, 6) %in% c(tongue, is_tongue) ~ "tongue",
                               substr(source_concept_code, 1, 5) %in% gum ~ "gum",
                               substr(source_concept_code, 1, 5) %in% c(mouth_floor, is_mouth_floor) ~ "mouth floor",
                               substr(source_concept_code, 1, 6) %in% c(mouth_floor, is_mouth_floor) ~ "mouth floor",
                               substr(source_concept_code, 1, 5) %in% c(palate, is_palate) ~ "palate",
                               substr(source_concept_code, 1, 6) %in% c(palate, is_palate) ~ "palate",
                               substr(source_concept_code, 1, 5) %in% c(other_mouth, is_other_mouth) ~ "other mouth",
                               substr(source_concept_code, 1, 6) %in% c(other_mouth, is_other_mouth) ~ "other mouth",
                               substr(source_concept_code, 1, 5) %in% parotid_gland ~ "parotid gland",
                               substr(source_concept_code, 1, 5) %in% salivary_glands ~ "salivary glands",
                               substr(source_concept_code, 1, 5) %in% tonsil ~ "tonsil",
                               substr(source_concept_code, 1, 5) %in% oropharynx ~ "oropharynx",
                               substr(source_concept_code, 1, 5) %in% nasopharynx ~ "nasopharynx",
                               substr(source_concept_code, 1, 5) %in% pyriform_sinus ~ "pyriform sinus",
                               substr(source_concept_code, 1, 5) %in% hypopharynx ~ "hypopharynx",
                               substr(source_concept_code, 1, 5) %in% other_head_and_neck ~ "other head and neck",
                               substr(source_concept_code, 1, 5) %in% c(esophagus, is_esophagus) ~ "esophagus",
                               substr(source_concept_code, 1, 6) %in% c(esophagus, is_esophagus) ~ "esophagus",
                               substr(source_concept_code, 1, 5) %in% c(stomach, is_stomach) ~ "stomach",
                               substr(source_concept_code, 1, 5) %in% small_intestine ~ "small intestine",
                               substr(source_concept_code, 1, 5) %in% appendix ~ "appendix",
                               substr(source_concept_code, 1, 5) %in% c(colon, is_colon) ~ "colon",
                               substr(source_concept_code, 1, 5) %in% rectosigmoid_junction ~ "rectosigmoid junction",
                               substr(source_concept_code, 1, 5) %in% c(rectum, is_rectum) ~ "rectum",
                               substr(source_concept_code, 1, 5) %in% c(anus, is_anus) ~ "anus",
                               substr(source_concept_code, 1, 5) %in% liver ~ "liver",
                               substr(source_concept_code, 1, 3) %in% liver ~ "liver",
                               substr(source_concept_code, 1, 5) %in% gallbladder ~ "gallbladder",
                               substr(source_concept_code, 1, 5) %in% pancreas ~ "pancreas",
                               substr(source_concept_code, 1, 5) %in% nasal_ear ~ "nasal cavity and middle ear",
                               substr(source_concept_code, 1, 5) %in% accessory_sinuses ~ "accessory sinuses",
                               substr(source_concept_code, 1, 5) %in% c(larynx, is_larynx) ~ "larynx",
                               substr(source_concept_code, 1, 5) %in% c(trachea, is_trachea) ~ "trachea",
                               substr(source_concept_code, 1, 5) %in% c(lung, is_lung) ~ "lung and bronchus",
                               substr(source_concept_code, 1, 5) %in% thymus ~ "thymus",
                               substr(source_concept_code, 1, 5) %in% heart_other ~ "heart and other",
                               substr(source_concept_code, 1, 5) %in% bone ~ "bone and cartilage",
                               substr(source_concept_code, 1, 5) %in% c(melanoma, is_melanoma) ~ "melanoma",
                               substr(source_concept_code, 1, 5) %in% c(non_melanoma_skin, is_non_melanoma_skin) ~ "non-melanoma skin",
                               substr(source_concept_code, 1, 5) %in% mesothelioma ~ "mesothelioma",
                               substr(source_concept_code, 1, 5) %in% Kaposi_sarcoma ~ "Kaposi's sarcoma",
                               substr(source_concept_code, 1, 5) %in% peripheral_nervous ~ "peripheral nervous system",
                               substr(source_concept_code, 1, 5) %in% retroperitoneum_peritoneum ~ "retroperitoneum and peritoneum",
                               substr(source_concept_code, 1, 5) %in% other_soft ~ "other soft tissue",
                               substr(source_concept_code, 1, 5) %in% c(breast, is_breast) ~ "breast",
                               substr(source_concept_code, 1, 5) %in% c(vulva, is_vulva) ~ "vulva",
                               substr(source_concept_code, 1, 5) %in% c(vagina, is_vagina) ~ "vagina",
                               substr(source_concept_code, 1, 5) %in% c(cervix, is_cervix) ~ "cervix",
                               substr(source_concept_code, 1, 5) %in% other_uterus ~ "other uterus",
                               substr(source_concept_code, 1, 5) %in% c(endometrium, is_endometrium) ~ "endometrium",
                               substr(source_concept_code, 1, 5) %in% ovary ~ "ovary",
                               substr(source_concept_code, 1, 5) %in% placenta ~ "placenta",
                               substr(source_concept_code, 1, 5) %in% c(penis, is_penis) ~ "penis",
                               substr(source_concept_code, 1, 5) %in% c(prostate, is_prostate) ~ "prostate",
                               substr(source_concept_code, 1, 5) %in% testis ~ "testis",
                               substr(source_concept_code, 1, 5) %in% kidney ~ "kidney",
                               substr(source_concept_code, 1, 5) %in% renal_pelvis ~ "renal pelvis",
                               substr(source_concept_code, 1, 5) %in% ureter ~ "ureter",
                               substr(source_concept_code, 1, 5) %in% c(bladder, is_bladder) ~ "bladder",
                               substr(source_concept_code, 1, 5) %in% c(eye, is_eye) ~ "eye",
                               substr(source_concept_code, 1, 5) %in% meninges ~ "meninges",
                               substr(source_concept_code, 1, 5) %in% brain ~ "brain",
                               substr(source_concept_code, 1, 5) %in% other_central_nervous ~ "other central nervous",
                               substr(source_concept_code, 1, 5) %in% thyroid_gland ~ "thyroid gland",
                               substr(source_concept_code, 1, 5) %in% adrenal_gland ~ "adrenal gland",
                               substr(source_concept_code, 1, 5) %in% Hodgkin_lymphoma ~ "Hodgkin's lymphoma",
                               substr(source_concept_code, 1, 5) %in% follicular_lymphoma ~ "follicular lymphoma",
                               substr(source_concept_code, 1, 5) %in% non_follicular_lymphoma ~ "non-follicular lymphoma",
                               substr(source_concept_code, 1, 5) %in% T_NK_lymphoma ~ "mature T and NK-cell lymphoma",
                               substr(source_concept_code, 1, 5) %in% other_non_Hodgkin_lymphoma ~ "other non-Hodgkin's lymphoma",
                               substr(source_concept_code, 1, 5) %in% other_T_NK_lymphoma ~ "other T and NK-cell lymphoma",
                               substr(source_concept_code, 1, 5) %in% immunoproliferative ~ "malignant immunoproliferative disease",
                               substr(source_concept_code, 1, 5) %in% multiple_myeloma ~ "multiple myeloma",
                               substr(source_concept_code, 1, 5) %in% lymphoid_leukemia ~ "lymphoid leukemia",
                               substr(source_concept_code, 1, 5) %in% myeloid_leukemia ~ "myeloid leukemia",
                               substr(source_concept_code, 1, 3) %in% myeloid_leukemia ~ "myeloid leukemia",
                               substr(source_concept_code, 1, 5) %in% monocytic_leukemia ~ "monocytic leukemia",
                               substr(source_concept_code, 1, 5) %in% other_leukemia ~ "other leukemia",
                               TRUE ~ NA_character_))

is_codes = c(is_tongue,is_mouth_floor,is_palate,is_other_mouth,is_esophagus,is_stomach,is_colon,is_rectum,is_anus,is_larynx,
            is_trachea,is_lung,is_melanoma,is_non_melanoma_skin,is_breast,is_cervix,is_endometrium,is_vulva,is_vagina,
            is_penis,is_prostate,is_bladder,is_eye)

data_first_condition = data_first_condition %>% 
mutate(invasive = ifelse(source_concept_code %in% is_codes,0,1))

## Create variables that give the order of new ICD diagnosis codes and sites by date for individuals with multiple diagnoses

In [ ]:
add_code_order_variable <- function(X) {
  ordered_df <- X %>%
    group_by(person_id) %>%
    arrange(diagnosis_date) %>%
    mutate(diagnosis_code_order = row_number()) %>%
    ungroup() %>%
    arrange(person_id)
  
  return(ordered_df)
}
add_site_order_variable <- function(X) {
  ordered_df <- X %>%
    group_by(person_id,cancer_site) %>%
    arrange(diagnosis_date) %>%
    mutate(diagnosis_site_order = row_number()) %>%
    ungroup() %>%
    arrange(person_id)
  
  return(ordered_df)
}
data_first_condition = add_code_order_variable(data_first_condition)
data_first_condition = add_site_order_variable(data_first_condition)

## Get DNA sample collection information

In [ ]:
sample_collection_data = fread("./sample_QC/genomic_metrics.tsv")

### Summary of relevant QC information

In [ ]:
table(sample_collection_data$sample_source,deparse.level=2,useNA="ifany")
table(sample_collection_data$sample_source,deparse.level=2,useNA="ifany")%>%
prop.table() %>%
round(3)
table(sample_collection_data$sample_source,sample_collection_data$site_id,deparse.level=2,useNA="ifany")
table(sample_collection_data$sample_source,sample_collection_data$site_id,deparse.level=2,useNA="ifany") %>%
prop.table(margin=2) %>%
round(3)

In [ ]:
sample_collection_data %>%
group_by(sample_source) %>%
summarize(first_q_mean_depth = quantile(mean_coverage,0.25) %>% round(3),
          median_mean_depth = median(mean_coverage) %>% round(3),
          third_q_mean_depth = quantile(mean_coverage,0.75) %>% round(3))

ggplot(sample_collection_data,aes(y=mean_coverage,x=sample_source))+
geom_violin()+
geom_boxplot(width=0.1) +
theme_minimal()

In [ ]:
sample_collection_data %>%
group_by(sample_source) %>%
summarize(first_q_mean_coverage = quantile(genome_coverage,0.25) %>% round(3),
          median_mean_coverage = median(genome_coverage) %>% round(3),
          third_q_mean_coverage = quantile(genome_coverage,0.75) %>% round(3))

ggplot(sample_collection_data,aes(y=genome_coverage,x=sample_source))+
geom_violin()+
geom_boxplot(width=0.1) +
theme_minimal()

### Prepare new variables

In [ ]:
sample_collection_data = sample_collection_data %>%
mutate(saliva_DNA = ifelse(sample_source == "SALIVA",1,0),
      genetic_sex = case_when(dragen_sex_ploidy == "XY" ~ "Male",
                             dragen_sex_ploidy == "XX" ~ "Female",
                             !dragen_sex_ploidy %in% c("XY","XX") ~ "other_or_intersex"),
      sample_date = as.IDate(biosample_collection_date))

### Extract relevant fields

In [ ]:
sample_data = sample_collection_data %>% select(c(research_id,site_id,saliva_DNA,genetic_sex,sample_date))
colnames(sample_data) = c("person_id","site_id","saliva_DNA","genetic_sex","sample_date")

## Create a numeric birth date variable and filter the demographics data set

In [ ]:
demographics$birth_date = as.IDate(demographics$date_of_birth)

In [ ]:
demographics = demographics %>% select(c(person_id,sex_at_birth,birth_date))

## Merge the conditions, demographics, and survey data sets 

In [ ]:
data = merge(demographics,data_first_condition, by="person_id", all.x = TRUE)
data = merge(data, sample_data, by = "person_id")

In [ ]:
nrow(demographics)

In [ ]:
length(unique(data$person_id))

## Create case indicator variable

In [ ]:
data$case = ifelse(is.na(data$standard_concept_name)==FALSE,1,0)

## Create age variables

In [ ]:
data = data %>% mutate(age_at_sample = round(as.numeric(difftime(sample_date, birth_date)/365.2425),1),
                      age_at_diagnosis = round(as.numeric(difftime(diagnosis_date, birth_date)/365.2425),1),
                      current_age = round(as.numeric(difftime(as.IDate(Sys.time()), birth_date)/365.2425),1))

## Create an incident case indicator variable

In [ ]:
data = data %>% mutate(incident = ifelse(age_at_diagnosis > age_at_sample,1,0))

## Load ancestry data and merge 

In [ ]:
ancestry <- read.delim("./ancestry/ancestry_preds.tsv")
ancestry = ancestry %>% select("research_id","ancestry_pred")
colnames(ancestry) = c("person_id","Assigned_Superpopulation")
ancestry = ancestry %>% mutate(Assigned_Superpopulation = toupper(Assigned_Superpopulation))
final = merge(data, ancestry, all.x=TRUE)

## Save the combined data set

In [ ]:
# # Replace 'test.csv' with THE NAME of the file you're going to store in the bucket (don't delete the quotation marks)
# destination_filename <- 'cleaned_AoU_WGS_cancer_data.csv'

# ########################################################################
# ##
# ################# DON'T CHANGE FROM HERE ###############################
# ##
# ########################################################################

# # store the dataframe in current workspace
# write_excel_csv(final, destination_filename)

# # Copy the file from current workspace to the bucket
# system(paste0("gsutil cp ./", destination_filename, " ",
#               file.path(Sys.getenv('WORKSPACE_BUCKET'),
#   "AoU_cancer",
#   "phenotypes",
#   "cleaned_AoU_WGS_cancer_data.csv")), intern=T)

# # Check if file is in the bucket
# system(paste0("gsutil ls ", file.path(Sys.getenv('WORKSPACE_BUCKET'),
#   "AoU_cancer",
#   "phenotypes",
#   "cleaned_AoU_WGS_cancer_data.csv")), intern=T)

# General cancer data summary

## Prepare data to summarize

In [ ]:
summary = final %>% mutate(Age_Range_S = case_when(
  age_at_sample<30 ~ "18-29",
  age_at_sample>=30 &  age_at_sample<40 ~ "30-39",
  age_at_sample>=40 &  age_at_sample<50 ~ "40-49",
  age_at_sample>=50 &  age_at_sample<60 ~ "50-59",
  age_at_sample>=60 &  age_at_sample<70 ~ "60-69",
  age_at_sample>=70 &  age_at_sample<80 ~ "70-79",
  age_at_sample>=80 &  age_at_sample<90 ~ "80-89",
  age_at_sample>=90 ~ "90+"))

summary = summary %>% mutate(Age_Range_D = case_when(
  age_at_diagnosis<10 ~ "<9",
  age_at_diagnosis>=10 &  age_at_diagnosis<20 ~ "10-19",
  age_at_diagnosis>=20 &  age_at_diagnosis<30 ~ "20-29",
  age_at_diagnosis>=30 &  age_at_diagnosis<40 ~ "30-39",
  age_at_diagnosis>=40 &  age_at_diagnosis<50 ~ "40-49",
  age_at_diagnosis>=50 &  age_at_diagnosis<60 ~ "50-59",
  age_at_diagnosis>=60 &  age_at_diagnosis<70 ~ "60-69",
  age_at_diagnosis>=70 &  age_at_diagnosis<80 ~ "70-79",
  age_at_diagnosis>=80 &  age_at_diagnosis<90 ~ "80-89",
  age_at_diagnosis>=90 ~ "90+"))

summary = summary %>%  mutate(value = 1) %>%
  pivot_wider(names_from = Assigned_Superpopulation,
              values_from = value,
              values_fill = list(value = 0))
  

## Subset case and control data sets ##
case_data = summary %>% filter(case==1)
control_data = summary %>% filter(case==0)

total = nrow(case_data %>% distinct(person_id))

## Case data summary

### Case counts, incidence, and sex by cancer type 

In [ ]:
# install.packages("reactable")
library(reactable)
case_data %>% group_by(cancer_site) %>% 
    distinct(person_id, .keep_all = TRUE) %>%
  summarise(N = n(),
                                                   percent_of_cases = paste0(((n()/total)*100) %>% round(2),"%"),
                                                   incident_case = paste0(sum(incident==1)," (",((sum(incident==1)/n())*100)%>%round(1),"%)"),
                                                   prevalent_case = paste0(sum(incident==0)," (",((sum(incident==0)/n())*100)%>%round(1),"%)"),
                                                   
                                                   female = paste0(sum(sex_at_birth=="Female")," (",((sum(sex_at_birth=="Female")/n())*100)%>%round(1),"%)"),
                                                   male = paste0(sum(sex_at_birth=="Male")," (",((sum(sex_at_birth=="Male")/n())*100)%>%round(1),"%)")) %>% 
  reactable(filterable = TRUE, defaultPageSize = 100, highlight = TRUE, striped = TRUE)

### Case counts, incidence, and sex by cancer type (invasive only)

In [ ]:
# install.packages("reactable")
library(reactable)
case_data %>% group_by(cancer_site) %>% 
  filter(invasive == 1) %>%
    distinct(person_id, .keep_all = TRUE) %>%
  summarise(N = n(),
                                                   percent_of_cases = paste0(((n()/total)*100) %>% round(2),"%"),
                                                   incident_case = paste0(sum(incident==1)," (",((sum(incident==1)/n())*100)%>%round(1),"%)"),
                                                   prevalent_case = paste0(sum(incident==0)," (",((sum(incident==0)/n())*100)%>%round(1),"%)"),
                                                   
                                                   female = paste0(sum(sex_at_birth=="Female")," (",((sum(sex_at_birth=="Female")/n())*100)%>%round(1),"%)"),
                                                   male = paste0(sum(sex_at_birth=="Male")," (",((sum(sex_at_birth=="Male")/n())*100)%>%round(1),"%)")) %>% 
  reactable(filterable = TRUE, defaultPageSize = 100, highlight = TRUE, striped = TRUE)

### Sample collection 10 year age ranges by cancer type

In [ ]:
case_data %>% group_by(cancer_site) %>% 
    distinct(person_id, .keep_all = TRUE) %>%
  summarise(
                                                   age18_29 = paste0(sum(Age_Range_S=="18-29")," (",((sum(Age_Range_S=="18-29")/n())*100)%>%round(1),"%)"),
                                                   age30_39 = paste0(sum(Age_Range_S=="30-39")," (",((sum(Age_Range_S=="30-39")/n())*100)%>%round(1),"%)"),
                                                   age40_49 = paste0(sum(Age_Range_S=="40-49")," (",((sum(Age_Range_S=="40-49")/n())*100)%>%round(1),"%)"),
                                                   age50_59 = paste0(sum(Age_Range_S=="50-59")," (",((sum(Age_Range_S=="50-59")/n())*100)%>%round(1),"%)"),
                                                   age60_69 = paste0(sum(Age_Range_S=="60-69")," (",((sum(Age_Range_S=="60-69")/n())*100)%>%round(1),"%)"),
                                                   age70_79 = paste0(sum(Age_Range_S=="70-79")," (",((sum(Age_Range_S=="70-79")/n())*100)%>%round(1),"%)"),
                                                   age80_89 = paste0(sum(Age_Range_S=="80-89")," (",((sum(Age_Range_S=="80-89")/n())*100)%>%round(1),"%)"),
                                                   age90_plus = paste0(sum(Age_Range_S=="90+")," (",((sum(Age_Range_S=="90+")/n())*100)%>%round(1),"%)")) %>% reactable(filterable = TRUE, defaultPageSize = 10, highlight = TRUE, striped = TRUE)


### Diagnosis 10 year age ranges by cancer type

In [ ]:
case_data %>% group_by(cancer_site) %>%
    distinct(person_id, .keep_all = TRUE) %>%
  summarise(
                                                   age0_9 = paste0(sum(Age_Range_D=="<9")," (",((sum(Age_Range_D=="<9")/n())*100)%>%round(1),"%)"),
                                                   age10_19 = paste0(sum(Age_Range_D=="10-19")," (",((sum(Age_Range_D=="10-19")/n())*100)%>%round(1),"%)"),
                                                   age20_29 = paste0(sum(Age_Range_D=="20-29")," (",((sum(Age_Range_D=="20-29")/n())*100)%>%round(1),"%)"),
                                                   age30_39 = paste0(sum(Age_Range_D=="30-39")," (",((sum(Age_Range_D=="30-39")/n())*100)%>%round(1),"%)"),
                                                   age40_49 = paste0(sum(Age_Range_D=="40-49")," (",((sum(Age_Range_D=="40-49")/n())*100)%>%round(1),"%)"),
                                                   age50_59 = paste0(sum(Age_Range_D=="50-59")," (",((sum(Age_Range_D=="50-59")/n())*100)%>%round(1),"%)"),
                                                   age60_69 = paste0(sum(Age_Range_D=="60-69")," (",((sum(Age_Range_D=="60-69")/n())*100)%>%round(1),"%)"),
                                                   age70_79 = paste0(sum(Age_Range_D=="70-79")," (",((sum(Age_Range_D=="70-79")/n())*100)%>%round(1),"%)"),
                                                   age80_89 = paste0(sum(Age_Range_D=="80-89")," (",((sum(Age_Range_D=="80-89")/n())*100)%>%round(1),"%)"),
                                                   age90_plus = paste0(sum(Age_Range_D=="90+")," (",((sum(Age_Range_D=="90+")/n())*100)%>%round(1),"%)")) %>% reactable(filterable = TRUE, defaultPageSize = 10, highlight = TRUE, striped = TRUE)


### Genetic ancestry by cancer type 

In [ ]:
case_data %>% group_by(cancer_site) %>%
    distinct(person_id, .keep_all = TRUE) %>%
  summarise(EUR = paste0(sum(EUR==1)," (",((sum(EUR==1)/n())*100)%>%round(1),"%)"),
                                                   AFR = paste0(sum(AFR==1)," (",((sum(AFR==1)/n())*100)%>%round(1),"%)"),
                                                   EAS = paste0(sum(EAS==1)," (",((sum(EAS==1)/n())*100)%>%round(1),"%)"),
                                                   AMR = paste0(sum(AMR==1)," (",((sum(AMR==1)/n())*100)%>%round(1),"%)"),
                                                   SAS = paste0(sum(SAS==1)," (",((sum(SAS==1)/n())*100)%>%round(1),"%)"),
                                               MID = paste0(sum(MID==1)," (",((sum(MID==1)/n())*100)%>%round(1),"%)")) %>%
  reactable(filterable = TRUE, defaultPageSize = 1000, highlight = TRUE, striped = TRUE)


### Genetic ancestry by cancer type (invasive only)

In [ ]:
case_data %>% group_by(cancer_site) %>%
    filter(invasive == 1) %>%
    distinct(person_id, .keep_all = TRUE) %>%
  summarise(EUR = paste0(sum(EUR==1)," (",((sum(EUR==1)/n())*100)%>%round(1),"%)"),
                                                   AFR = paste0(sum(AFR==1)," (",((sum(AFR==1)/n())*100)%>%round(1),"%)"),
                                                   EAS = paste0(sum(EAS==1)," (",((sum(EAS==1)/n())*100)%>%round(1),"%)"),
                                                   AMR = paste0(sum(AMR==1)," (",((sum(AMR==1)/n())*100)%>%round(1),"%)"),
                                                   SAS = paste0(sum(SAS==1)," (",((sum(SAS==1)/n())*100)%>%round(1),"%)"),
                                               MID = paste0(sum(MID==1)," (",((sum(MID==1)/n())*100)%>%round(1),"%)")) %>%
  reactable(filterable = TRUE, defaultPageSize = 1000, highlight = TRUE, striped = TRUE)


## Control data summary

In [ ]:
# install.packages("table1")
library(table1)
table1(~sex_at_birth+Age_Range_S+as.factor(EUR)+as.factor(AFR)+as.factor(EAS)+as.factor(AMR)+as.factor(SAS)+as.factor(MID),data = control_data) %>% knitr::kable()

# Age comparison by case type

In [1]:
library(pacman)

p_load(tidyverse,
       data.table,
       ggpubr,
       gridExtra,
       cowplot)

In [18]:
data = fread("./cleaned_AoU_WGS_cancer_data.csv")

In [6]:
names(data)

[1] "person_id"                     "sex_at_birth"                 
 [3] "birth_date"                    "standard_concept_name"        
 [5] "standard_concept_code"         "standard_vocabulary"          
 [7] "condition_start_datetime"      "condition_end_datetime"       
 [9] "condition_type_concept_name"   "stop_reason"                  
[11] "visit_occurrence_concept_name" "condition_source_value"       
[13] "source_concept_name"           "source_concept_code"          
[15] "source_vocabulary"             "condition_status_source_value"
[17] "condition_status_concept_name" "diagnosis_date"               
[19] "cancer_site"                   "invasive"                     
[21] "diagnosis_code_order"          "diagnosis_site_order"         
[23] "site_id"                       "saliva_DNA"                   
[25] "genetic_sex"                   "sample_date"                  
[27] "case"                          "age_at_sample"                
[29] "age_at_diagnosis"              "current_age"                  
[31] "incident"                      "Assigned_Superpopulation"

In [19]:
# Filter to only the first recorded diagnosis for each case and cases with ages at diagnosis#
data <- data %>%
  filter(is.na(age_at_diagnosis) == FALSE, cancer_site != "") %>%
  group_by(person_id) %>%
  filter(diagnosis_code_order == min(diagnosis_code_order)) %>%
  ungroup()

# Define incident cases #
data = data %>%
  mutate(type = case_when(incident == 0 ~ "Prevalent",
                          incident == 1 ~ "Incident"))

In [8]:
if (!dir.exists("./age_comparison/plots")) dir.create("./age_comparison/plots")

In [ ]:
## Plots of age at diagnosis by case type across cancer sites ##

if (!dir.exists("./age_comparison/plots")) dir.create("./age_comparison/plots")

plot_data <- data %>%
  group_by(cancer_site, type) %>%
  mutate(n_type = n()) %>%
  ungroup()

valid_sites <- plot_data %>%
  group_by(cancer_site) %>%
  summarise(total_n = n()) %>%
  filter(total_n > 500) %>%
  pull(cancer_site)

plot_data <- plot_data %>% filter(cancer_site %in% valid_sites)

mean_diff <- plot_data %>%
  group_by(cancer_site, type) %>%
  summarise(mean_age = mean(age_at_diagnosis, na.rm = TRUE), .groups = "drop") %>%
  tidyr::pivot_wider(names_from = type, values_from = mean_age) %>%
  mutate(age_diff = abs(`Incident` - `Prevalent`))   

site_order <- mean_diff %>%
  arrange(desc(age_diff)) %>%
  pull(cancer_site)

site_groups <- split(site_order, ceiling(seq_along(site_order)/4))

plots <- lapply(site_groups, function(group_sites) {
  
  subset_data <- plot_data %>% filter(cancer_site %in% group_sites)
  
  summary_tbl <- subset_data %>%
    group_by(cancer_site, type) %>%
    summarise(
      n = n(),
      mean_age = round(mean(age_at_diagnosis, na.rm = TRUE), 1),
      .groups = "drop"
    ) %>%
    tidyr::pivot_wider(
      names_from = type,
      values_from = c(n, mean_age),
      values_fill = 0
    )
  
  p <- ggplot(subset_data,
              aes(x = age_at_diagnosis, y = reorder(cancer_site, n_type, sum), color = type)) +
    geom_violin(trim = FALSE, alpha = 0.5) +
    geom_boxplot(width = 0.2,
                 position = position_dodge(width = 0.9),
                 outlier.size = 0.5) +
    theme_bw() +
    xlab("Age at diagnosis") +
    ylab("Cancer")
  tbl <- tableGrob(summary_tbl, rows = NULL)
  
  combined <- plot_grid(p, tbl, ncol = 1, rel_heights = c(3, 1))
  
  fname <- paste0("./age_comparison/plots/Age_dx_cancers_", paste(group_sites, collapse = "_"), ".png")
  ggsave(fname, combined, width = 12, height = 12, dpi = 300)
  
  return(combined)
})

In [ ]:
## Plots of age at diagnosis by case type across cancer sites ##

if (!dir.exists("./age_comparison/plots")) dir.create("./age_comparison/plots")

plot_data <- data %>%
  group_by(cancer_site, type) %>%
  mutate(n_type = n()) %>%
  ungroup()

valid_sites <- plot_data %>%
  group_by(cancer_site) %>%
  summarise(total_n = n()) %>%
  filter(total_n > 500) %>%
  pull(cancer_site)

plot_data <- plot_data %>% filter(cancer_site %in% valid_sites)

mean_diff <- plot_data %>%
  group_by(cancer_site, type) %>%
  summarise(mean_age = mean(age_at_sample, na.rm = TRUE), .groups = "drop") %>%
  tidyr::pivot_wider(names_from = type, values_from = mean_age) %>%
  mutate(age_diff = abs(`Incident` - `Prevalent`))   

site_order <- mean_diff %>%
  arrange(desc(age_diff)) %>%
  pull(cancer_site)

site_groups <- split(site_order, ceiling(seq_along(site_order)/4))

plots <- lapply(site_groups, function(group_sites) {
  
  subset_data <- plot_data %>% filter(cancer_site %in% group_sites)
  
  summary_tbl <- subset_data %>%
    group_by(cancer_site, type) %>%
    summarise(
      n = n(),
      mean_age = round(mean(age_at_diagnosis, na.rm = TRUE), 1),
      .groups = "drop"
    ) %>%
    tidyr::pivot_wider(
      names_from = type,
      values_from = c(n, mean_age),
      values_fill = 0
    )
  
  p <- ggplot(subset_data,
              aes(x = age_at_sample, y = reorder(cancer_site, n_type, sum), color = type)) +
    geom_violin(trim = FALSE, alpha = 0.5) +
    geom_boxplot(width = 0.2,
                 position = position_dodge(width = 0.9),
                 outlier.size = 0.5) +
    theme_bw() +
    xlab("Age at sample") +
    ylab("Cancer")
  tbl <- tableGrob(summary_tbl, rows = NULL)
  
  combined <- plot_grid(p, tbl, ncol = 1, rel_heights = c(3, 1))
  
  fname <- paste0("./age_comparison/plots/Age_sample_cancers_", paste(group_sites, collapse = "_"), ".png")
  ggsave(fname, combined, width = 12, height = 12, dpi = 300)
  
  return(combined)
})

In [20]:
## Tabular summary ##
data = data %>%
  group_by(cancer_site) %>%
  mutate(total_n = n()) %>%
  filter(total_n > 500)

summary_df <- data %>%
  group_by(cancer_site, type) %>%
  summarise(
    n = n(),
    mean_DxAge   = mean(age_at_diagnosis, na.rm = TRUE),
    sd_DxAge     = sd(age_at_diagnosis, na.rm = TRUE),
    mean_AgeSample = mean(age_at_sample, na.rm = TRUE),
    sd_AgeSample   = sd(age_at_sample, na.rm = TRUE),
    .groups = "drop"
  ) %>%
  pivot_wider(
    names_from = type,
    values_from = c(n, mean_DxAge, sd_DxAge, mean_AgeSample, sd_AgeSample),
    values_fill = 0
  ) %>%
  mutate(
    diff_DxAge      = mean_DxAge_Incident - mean_DxAge_Prevalent,    
    diff_AgeSample  = mean_AgeSample_Incident - mean_AgeSample_Prevalent
  )

## Add overall means (across both incident + prevalent cases)
overall_df <- data %>%
  group_by(cancer_site) %>%
  summarise(
    overall_mean_DxAge   = mean(age_at_diagnosis, na.rm = TRUE),
    overall_sd_DxAge     = sd(age_at_diagnosis, na.rm = TRUE),
    overall_mean_AgeSample = mean(age_at_sample, na.rm = TRUE),
    overall_sd_AgeSample   = sd(age_at_sample, na.rm = TRUE),
    .groups = "drop"
  )

## Merge and format
summary_df <- summary_df %>%
  left_join(overall_df, by = "cancer_site") %>%
  mutate(
    across(
      c(starts_with("mean_"), starts_with("sd_"), starts_with("overall_"), starts_with("diff_")),
      ~round(.x, 1)
    ),
    
    ## Combine means and SDs into formatted strings
    DxAge_Incident   = paste0(mean_DxAge_Incident, " (", sd_DxAge_Incident, ")"),
    DxAge_Prevalent  = paste0(mean_DxAge_Prevalent, " (", sd_DxAge_Prevalent, ")"),
    DxAge_Overall    = paste0(overall_mean_DxAge, " (", overall_sd_DxAge, ")"),
    
    AgeSample_Incident  = paste0(mean_AgeSample_Incident, " (", sd_AgeSample_Incident, ")"),
    AgeSample_Prevalent = paste0(mean_AgeSample_Prevalent, " (", sd_AgeSample_Prevalent, ")"),
    AgeSample_Overall   = paste0(overall_mean_AgeSample, " (", overall_sd_AgeSample, ")")
  ) %>%
  select(
    cancer_site,
    starts_with("n_"),
    DxAge_Incident, DxAge_Prevalent, DxAge_Overall, diff_DxAge,
    AgeSample_Incident, AgeSample_Prevalent, AgeSample_Overall, diff_AgeSample
  )

## Save output
fwrite(summary_df, "./age_comparison/AoU_ages_at_dx_and_sample_comparison.csv")